# Applying Various ML Models (RNN, LSTM, CNN, CNN→RNN, CNN→LSTM) 

## Step 1: Data Preprocessing

In [91]:
from __future__ import annotations
import numpy as np, pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
import tensorflow as tf
tf.config.run_functions_eagerly(False)  # ensure graph mode
from tensorflow.keras import layers, models, callbacks, optimizers
from tensorflow.keras.losses import Huber

In [92]:
# --- CONFIG (smart) ---
# What you want to do
FORECAST_MONTHS = 12          # set to 12 if you need 2025M1–M12
SEASONAL_PERIOD = 12          # monthly seasonality

# Windowing
LOOKBACK       = max(SEASONAL_PERIOD, 12)   # at least one seasonal cycle
LOOKBACK_PLAN  = LOOKBACK
HORIZON        = FORECAST_MONTHS            # align all models to same H
HORIZON_PLAN   = HORIZON

# Splits / training
VAL_FRAC   = 0.20      # chronological validation fraction
MIN_TRAIN  = 1         # min train windows safeguard (don’t touch)
MIN_VAL    = 1         # min val windows safeguard (don’t touch)
BATCH      = 16        # will be clamped to dataset size at runtime
EPOCHS     = 200
PATIENCE   = 20
LR         = 1e-3
SEED       = 42

# Products (keep in sync with your data)
PRODUCTS = ["iPhone","iPad","MacBook"]

# Tariff/elasticity params (OPTIONAL; used only if you apply tariff layer)
ELASTICITY   = {"iPhone": -1.2, "iPad": -0.8, "MacBook": -1.0}   # demand elasticity
PASS_THROUGH = {"iPhone":  0.7, "iPad":  0.6, "MacBook":  0.65}  # share of tariff passed to price
CHINA_SHARE  = {"iPhone":  0.6, "iPad":  0.5, "MacBook":  0.55}  # input share
# Build a horizon-length tariff path (e.g., constant 5% then 10% after month 6)
TARIFF_PATH  = {h: (0.05 if h <= 6 else 0.10) for h in range(1, HORIZON+1)}

# Seeding (TF + NumPy + Python)
import os, random, numpy as np, tensorflow as tf
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED); np.random.seed(SEED)
tf.keras.utils.set_random_seed(SEED)

# # ---- Runtime sanity helpers (optional) ----
# def _print_windowing_sanity(df):
#     """Call right after you make df_use. Helps avoid 'too few windows' surprises."""
#     from math import floor
#     d = df.copy()
#     # total months per scenario (approx; ignores missing months)
#     by_scen = d.groupby("scenario")[["Year","period"]].nunique().assign(
#         approx_months=lambda x: x["Year"]*12  # rough, but useful guard
#     )
#     print("\n[Sanity] Windowing targets")
#     print(f"  LOOKBACK={LOOKBACK}, HORIZON={HORIZON}, VAL_FRAC={VAL_FRAC}")
#     print("[Sanity] Per-scenario rough month counts:\n", by_scen[["approx_months"]])

def _auto_adjust_for_small_data(n_windows):
    """Optional: call inside your training script to soften configs when data is tiny."""
    global VAL_FRAC, BATCH, EPOCHS, PATIENCE
    if n_windows < 10:
        VAL_FRAC = min(VAL_FRAC, 0.15)   # keep more for train
        BATCH    = min(BATCH, max(1, n_windows//2))
        EPOCHS   = min(EPOCHS, 150)
        PATIENCE = min(PATIENCE, 15)
def _norm_prod(name: str) -> str:
    return str(name).strip()

In [93]:
def load_demand_excel(path):
    df = pd.read_csv(
        path,
        dtype={"period": int, "product_id": int, "product_name": str, "scenario": str, "region":str},
    )
    df.columns = [c.strip() for c in df.columns]
    # parse numbers like "90,022" (works even if Excel already loaded them as numeric)
    df["demand_units"] = (
        df["demand_units"].astype(str).str.replace(",", "", regex=False).astype(float)
    )
    df["product_name"] = df["product_name"].str.strip()
    df = df.sort_values(["scenario", "product_id", "period","region"]).reset_index(drop=True)
    return df


In [94]:
def _windows_per_scenario(T: int, L: int, H: int, R: int) -> int:
    """
    How many rolling windows can we extract from a time series of length T
    after we 'repeat' the series R times (data augmentation),
    using lookback L and forecast horizon H?

    Formula: max(0, (T*R) - L - H + 1)

    Why:
      - (T*R) is the effective length after repeating R times.
      - For each window, we need L history points and H future points.
      - The last valid window starts at index (T*R - L - H).
    """
    return max(0, (T * R) - L - H + 1)


# --- uses your existing make_df_use(...) and normalize_time_strict(...) ---

def _choose_min_repeat(df, lookback: int, horizon: int,
                       val_frac: float = 0.2, min_train_windows: int = 8) -> int:
    """
    Choose the smallest augmentation repeat R (1..20) that yields enough *training* windows.
    We compute T per scenario using a continuous month index (t_month), not raw 'period'.
    """
    d = make_df_use(df)
    d = normalize_time_strict(d)  # adds t_month per scenario (0,1,2,...)
    # number of months (rows) per scenario in time order
    T_per_scen = d.groupby("scenario")["t_month"].max().add(1).to_dict()

    for R in range(1, 21):
        N_tot = sum(_windows_per_scenario(Ts, lookback, horizon, R) for Ts in T_per_scen.values())
        n_val = max(1, int(round(N_tot * val_frac))) if N_tot > 1 else 0
        n_tr  = N_tot - n_val
        if n_tr >= min_train_windows:
            return R
    return 2  # fallback for very small datasets


def make_windows_from_tidy(df, lookback, horizon, augment_repeat=None,
                           products=None, val_frac=0.2, min_train_windows=8):
    """
    Build rolling windows on a *rectangular* panel indexed by continuous t_month.
    Targets are the first len(PRODUCTS) columns; extras (month one-hots, flags) come after.
    """
    if products is None:
        products = PRODUCTS
    m_out = len(products)

    # auto-choose augmentation
    if augment_repeat is None:
        augment_repeat = _choose_min_repeat(df, lookback, horizon, val_frac, min_train_windows)

    # normalize & time-index
    d = make_df_use(df)
    d = normalize_time_strict(d)  # t_month added
    Xs, ys, per_scen_summary = [], [], []

    # build per-scenario rectangular panel on t_month
    for scen, g in d.groupby("scenario", sort=True):
        # sum regions if any already handled in make_df_use
        # pivot by *t_month* to avoid duplicates from repeating calendar months
        pv = (g.pivot_table(index="t_month", columns="product_name",
                            values="demand_units", aggfunc="sum")
                .reindex(columns=products))
        # handle gaps
        pv = pv.ffill().bfill()  # or pv.fillna(0.0)
        # --- exogenous features derived from calendar month (1..12) ---
        # month number from (t_month % 12) + 1
        month_num = (pv.index.values % 12) + 1
        month_oh = pd.get_dummies(month_num)
        # consistent names m_01..m_12 (avoid m_2..m_12 vs m_02..m_12 mismatches)
        month_oh.columns = [f"m_{i:02d}" for i in month_oh.columns]
        exo = month_oh.copy()
        exo["flag_launch"]  = (month_num == 9).astype(float)
        exo["flag_holiday"] = np.isin(month_num, [10, 11, 12]).astype(float)

        arr_prod = pv.values.astype("float32")             # (T, m_out)
        arr_exo  = exo.values.astype("float32")            # (T, n_exo)
        series   = np.concatenate([arr_prod, arr_exo], 1)  # (T, m_out + n_exo)

        # repeat contiguously R times (simple augmentation)
        series = np.vstack([series for _ in range(augment_repeat)])
        T_eff, m_in = series.shape
        N_here = T_eff - lookback - horizon + 1
        per_scen_summary.append((scen, T_eff, max(0, N_here)))
        if N_here <= 0:
            continue

        for start in range(N_here):
            end = start + lookback
            Xs.append(series[start:end, :])                         # (L, m_in)
            ys.append(series[end:end+horizon, :m_out].reshape(-1))  # (H*m_out,)

    if len(Xs) < 2:
        diag = "\n".join([f"  • scen='{s}': T_eff={tt}, windows={wh}" for s,tt,wh in per_scen_summary])
        raise ValueError(
            f"No/too few windows. lookback={lookback}, horizon={horizon}, repeat={augment_repeat}\n{diag}"
        )

    return np.asarray(Xs, dtype="float32"), np.asarray(ys, dtype="float32")


def naive_same_month_last_year(df, scenario, horizon, lookback):
    """
    Copy the same calendar month from *last year* using t_month indexing.
    If the history is too short, repeat the last 12-month profile.
    Returns an array of shape (H, m_out).
    """
    d = make_df_use(df)
    d = normalize_time_strict(d)
    g = d[d["scenario"] == scenario].copy()
    pv = (g.pivot_table(index="t_month", columns="product_name",
                        values="demand_units", aggfunc="sum")
            .reindex(columns=PRODUCTS)
            .sort_index())
    pv = pv.ffill().bfill()
    mat = pv.values.astype("float32")  # (T, m_out)
    T, m = mat.shape
    if T < 12 + horizon:
        # fallback: tile last 12
        last12 = (mat[-12:, :] if T >= 12 else np.tile(mat[-1:, :], (12, 1)))
        reps = int(np.ceil(horizon / 12))
        return np.vstack([last12] * reps)[:horizon, :]

    yhat = []
    for h in range(1, horizon + 1):
        tgt_idx = T - lookback + h - 1
        src_idx = tgt_idx - 12
        src_idx = np.clip(src_idx, 0, T - 1)
        yhat.append(mat[src_idx, :])
    return np.vstack(yhat)

# Splits the pooled windows into train/validation by keeping order and taking the last n_val windows as validation.This is time-aware (not random) and avoids shuffling, which is correct for time series.
def split_windows(X, y, val_frac=VAL_FRAC, min_train=1, min_val=1):
    """
    Chronological (time-aware) split of pooled windows.
    Keeps the last n_val windows for validation.
    Guarantees at least `min_train` train and `min_val` validation windows
    if possible. If data are too short, still returns valid arrays.
    """
    N = X.shape[0]
    if N < 2:
        # Not enough windows to split; all go to training
        print(f"[split_windows] Only {N} window(s) available — using all for training.")
        return X, y, X[:0], y[:0]

    # Compute validation count safely
    n_val = max(min_val, int(round(N * val_frac)))
    n_val = min(n_val, N - min_train)  # leave enough for training
    if n_val <= 0:
        n_val = 1 if N > 1 else 0

    # Chronological split (no shuffle)
    Xtr, ytr = X[:N - n_val], y[:N - n_val]
    Xval, yval = X[N - n_val:], y[N - n_val:]
    return Xtr, ytr, Xval, yval



## Step 2: Applying DL Models

In [95]:
# ---------------------------------------------------
# Canonical dataframe utilities to standardize schema
# ---------------------------------------------------
# def make_df_use(df):
#     """Unify column names; sum regions if present; add default scenario if missing"""
#     Y  = "Year" if "Year" in df.columns else "year"
#     P  = "period" if "period" in df.columns else ("month" if "month" in df.columns else "Month")
#     PN = "product_name" if "product_name" in df.columns else ("product" if "product" in df.columns else "product_id")
#     V  = ("demand_units" if "demand_units" in df.columns else
#           ("demand" if "demand" in df.columns else
#            ("qty" if "qty" in df.columns else "quantity")))
#     d = df.copy()
#     if "region" in d.columns:
#         d = d.groupby([Y, P, PN], as_index=False)[V].sum()
#     if "scenario" not in d.columns:
#         d["scenario"] = "Baseline_Seasonal"
#     d = d.rename(columns={Y:"Year", P:"period", PN:"product_name", V:"demand_units"})
#     d["Year"]   = d["Year"].astype(int)
#     d["period"] = d["period"].astype(int)
#     return d

def make_df_use(df):
    """Unify column names; sum regions if present; add default scenario if missing"""
    Y  = "Year"
    P  = "period" 
    PN = "product_name"
    V  = "demand_units"
    scenario="scenario"
    d = df.copy()
    d = d.groupby([Y, P, PN,scenario], as_index=False)[V].sum()
    d["Year"]   = d["Year"].astype(int)
    d["period"] = d["period"].astype(int)
    return d


def normalize_time_strict(d, sc="scenario", yc="Year", mc="period"):
    """Create a continuous month index per scenario (t_month)"""
    d = d.copy()
    d[yc] = d[yc].astype(int)
    d[mc] = d[mc].astype(int)
    months = (d[[sc, yc, mc]].drop_duplicates()
              .sort_values([sc, yc, mc]).reset_index(drop=True))
    months["t_month"] = months.groupby(sc).cumcount()
    return d.merge(months, on=[sc, yc, mc], how="left")

def panel_for_scenario_full(df, scen, products=PRODUCTS):
    """
    Build a rectangular (t_month × product) panel for one scenario.
    Works whether df already has t_month or only Year/period.
    """
    d = make_df_use(df)  # ensures Year/period/product_name/demand_units/scenario
    if "t_month" not in d.columns:
        d = normalize_time_strict(d)   # create t_month from Year/period per scenario

    g = d[d["scenario"] == scen].copy()
    if g.empty:
        raise ValueError(f"Scenario '{scen}' not found in dataframe.")

    # guard against dup rows for same (t_month, product)
    g = (g.groupby(["t_month", "product_name"], as_index=False)["demand_units"]
           .sum())

    pv = (g.pivot(index="t_month", columns="product_name", values="demand_units")
            .reindex(columns=products)
            .sort_index())

    # keep panel rectangular for windowing
    pv = pv.ffill()
    return pv


# -------------------
# Model architectures
# -------------------
def build_rnn(L, m, H, width=64, depth=1, dropout=0.2):
    x = layers.Input(shape=(L, m))
    h = x
    for _ in range(depth):
        h = layers.SimpleRNN(width, return_sequences=True)(h)
        h = layers.Dropout(dropout)(h)
    h = layers.Flatten()(h)
    out = layers.Dense(H * len(PRODUCTS))(h)
    model = models.Model(x, out)
    model.compile(optimizer=optimizers.Adam(LR), loss=Huber(delta=1.0))
    return model

def build_lstm(L, m, H, width=64, depth=1, dropout=0.2):
    x = layers.Input(shape=(L, m))
    h = x
    for i in range(depth):
        h = layers.LSTM(width, return_sequences=(i < depth-1))(h)
        h = layers.Dropout(dropout)(h)
    if depth == 1:
        h = layers.Flatten()(layers.RepeatVector(1)(h))
    else:
        h = layers.Flatten()(h)
    out = layers.Dense(H * len(PRODUCTS))(h)
    model = models.Model(x, out)
    model.compile(optimizer=optimizers.Adam(LR), loss=Huber(delta=1.0))
    return model

def build_cnn(L, m, H, width=64, depth=2, dropout=0.2):
    x = layers.Input(shape=(L, m))
    h = x
    for _ in range(depth):
        h = layers.Conv1D(filters=width, kernel_size=3, padding="causal", activation="relu")(h)
        h = layers.Dropout(dropout)(h)
    h = layers.GlobalAveragePooling1D()(h)
    out = layers.Dense(H * len(PRODUCTS))(h)
    model = models.Model(x, out)
    model.compile(optimizer=optimizers.Adam(LR), loss=Huber(delta=1.0))
    return model

def build_cnn_rnn(L, m, H, width=64, depth_cnn=2, rnn_units=64, dropout=0.2):
    x = layers.Input(shape=(L, m))
    h = x
    for _ in range(depth_cnn):
        h = layers.Conv1D(filters=width, kernel_size=3, padding="causal", activation="relu")(h)
        h = layers.Dropout(dropout)(h)
    h = layers.LSTM(rnn_units)(h)
    out = layers.Dense(H * len(PRODUCTS))(h)
    model = models.Model(x, out)
    model.compile(optimizer=optimizers.Adam(LR), loss=Huber(delta=1.0))
    return model

def build_cnn_lstm(L, m, H, width=64, depth_cnn=2, lstm_units=64, dropout=0.2):
    x = layers.Input(shape=(L, m))
    h = x
    for _ in range(depth_cnn):
        h = layers.Conv1D(filters=width, kernel_size=5, padding="causal", activation="relu")(h)
        h = layers.MaxPooling1D(pool_size=2)(h)
        h = layers.Dropout(dropout)(h)
    h = layers.LSTM(lstm_units)(h)
    out = layers.Dense(H * len(PRODUCTS))(h)
    model = models.Model(x, out)
    model.compile(optimizer=optimizers.Adam(LR), loss=Huber(delta=1.0))
    return model

MODEL_BUILDERS = {
    "rnn":       build_rnn,
    "lstm":      build_lstm,
    "cnn":       build_cnn,
    "cnn_rnn":   build_cnn_rnn,
    "cnn_lstm":  build_cnn_lstm,
}

# ------------------
# Error metrics
# ------------------
def rmse_vec(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float).ravel()
    y_pred = np.asarray(y_pred, dtype=float).ravel()
    return float(np.sqrt(np.mean((y_true - y_pred)**2)))

def smape_vec(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    denom  = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    with np.errstate(divide="ignore", invalid="ignore"):
        frac = np.abs(y_true - y_pred) / np.maximum(denom, 1e-12)
    return float(np.nanmean(frac) * 100.0)


# ---------- SAFE SPLITTER ----------
def split_windows_safe(X, y, val_frac=0.2, min_train=1, min_val=1):
    """Chronological split with safety for tiny datasets."""
    N = X.shape[0]
    if N < min_train + min_val:
        if N >= 2:
            split = N - 1
            Xtr, ytr = X[:split], y[:split]
            Xval, yval = X[split:], y[split:]
        else:
            Xtr, ytr = X, y
            Xval, yval = X[:0], y[:0]
        return Xtr, ytr, Xval, yval
    split = max(min_train, int(round(N * (1.0 - val_frac))))
    split = min(split, N - min_val)
    return X[:split], y[:split], X[split:], y[split:]

# ---------- UPDATED MODEL ZOO ----------
def run_model_zoo(df, lookback=LOOKBACK, horizon=HORIZON, augment_repeat=None, USE_Y_SCALING=True):
    X_raw, y_raw = make_windows_from_tidy(df, lookback, horizon, augment_repeat)  # X:(N,L,m_in)  y:(N,H*m_out)
    L, m_in = X_raw.shape[1], X_raw.shape[2]
    m_out   = len(PRODUCTS)
    assert y_raw.shape[1] % m_out == 0, "y must be shaped (N, H*m_out)"
    H  = y_raw.shape[1] // m_out

    Xtr_raw, ytr_raw, Xval_raw, yval_raw = split_windows_safe(X_raw, y_raw, val_frac=VAL_FRAC, min_train=1, min_val=1)
    n_tr, n_val = len(Xtr_raw), len(Xval_raw)
    if n_tr == 0:
        raise ValueError("No training windows created. Reduce LOOKBACK/HORIZON or add data.")
    has_val = n_val > 0


    x_scaler = StandardScaler().fit(Xtr_raw.reshape(-1, m_in))
    Xtr  = (Xtr_raw - x_scaler.mean_) / x_scaler.scale_
    Xval = (Xval_raw - x_scaler.mean_) / x_scaler.scale_ if has_val else Xval_raw

    if USE_Y_SCALING:
        ytr_mat  = ytr_raw.reshape(-1, H, m_out).reshape(-1, m_out)
        from sklearn.preprocessing import StandardScaler as _SS
        y_scaler = _SS().fit(ytr_mat)
        ytr = ((ytr_mat - y_scaler.mean_) / y_scaler.scale_).reshape(-1, H*m_out)
        if has_val:
            yval_mat = yval_raw.reshape(-1, H, m_out).reshape(-1, m_out)
            yval = ((yval_mat - y_scaler.mean_) / y_scaler.scale_).reshape(-1, H*m_out)
        else:
            yval = yval_raw
    else:
        y_scaler = None
        ytr, yval = ytr_raw, yval_raw

    def build_callbacks(has_val):
        if has_val:
            return [
                callbacks.EarlyStopping(monitor="val_loss", patience=PATIENCE, restore_best_weights=True),
                callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=max(8, PATIENCE//2), min_lr=1e-5),
            ]
        else:
            return [
                callbacks.EarlyStopping(monitor="loss", patience=PATIENCE, restore_best_weights=True),
                callbacks.ReduceLROnPlateau(monitor="loss", factor=0.5, patience=max(8, PATIENCE//2), min_lr=1e-5),
            ]

    eff_batch = max(1, min(BATCH, len(Xtr)))
    results, best = [], {"kind": None, "val_smape": 1e9, "model": None}
    for kind in ["rnn","lstm","cnn","cnn_rnn","cnn_lstm"]:
        mdl = MODEL_BUILDERS[kind](L, m_in, H)
        cbs = build_callbacks(has_val)
        if has_val:
            mdl.fit(Xtr, ytr, validation_data=(Xval, yval), epochs=EPOCHS, batch_size=eff_batch, verbose=0, callbacks=cbs)
            yhat_val = mdl.predict(Xval, verbose=0)
            if y_scaler is not None:
                yhat_val_mat = yhat_val.reshape(-1, H, m_out).reshape(-1, m_out)
                yval_mat     = yval.reshape(-1, H, m_out).reshape(-1, m_out)
                yhat_eval = yhat_val_mat * y_scaler.scale_ + y_scaler.mean_
                yval_eval = yval_mat     * y_scaler.scale_ + y_scaler.mean_
            else:
                yhat_eval, yval_eval = yhat_val, yval
        else:
            mdl.fit(Xtr, ytr, epochs=EPOCHS, batch_size=eff_batch, verbose=0, callbacks=cbs)
            yhat_tr = mdl.predict(Xtr, verbose=0)
            if y_scaler is not None:
                yhat_tr_mat = yhat_tr.reshape(-1, H, m_out).reshape(-1, m_out)
                ytr_mat2    = ytr.reshape(-1, H, m_out).reshape(-1, m_out)
                yhat_eval = yhat_tr_mat * y_scaler.scale_ + y_scaler.mean_
                yval_eval = ytr_mat2    * y_scaler.scale_ + y_scaler.mean_
            else:
                yhat_eval, yval_eval = yhat_tr, ytr

        sc = {
            "kind": kind,
            "val_rmse": float(np.sqrt(np.mean((yval_eval - yhat_eval)**2))),
            "val_smape": float(np.mean(
                np.abs(yval_eval - yhat_eval) /
                np.maximum((np.abs(yval_eval)+np.abs(yhat_eval))/2.0, 1e-8)
            )*100.0)
        }
        results.append(sc)
        if sc["val_smape"] < best["val_smape"]:
            best.update({"kind": kind, "val_smape": sc["val_smape"], "model": mdl})

    return best["model"], best["kind"], x_scaler, y_scaler, L, m_in, H, pd.DataFrame(results).sort_values("val_smape")

# -------------------------------------------------------
# Optional: short fine-tune on selected scenarios
# -------------------------------------------------------
def finetune_on_scenarios(model, df, scenarios, lookback, horizon,
                          x_scaler, y_scaler, epochs=3, batch_size=32,
                          augment_repeat=1, min_train_windows=2):
    df_sub = make_df_use(df)
    df_sub = df_sub[df_sub["scenario"].isin(scenarios)].copy()
    rep = int(augment_repeat if augment_repeat is not None else 1)
    while rep >= 1:
        try:
            X_raw, y_raw = make_windows_from_tidy(df_sub, lookback, horizon, augment_repeat=rep)
            break
        except ValueError as e:
            if "No/too few windows" in str(e) and rep > 1:
                print(f"[Fine-tune] Not enough windows with repeat={rep}. Trying repeat={rep-1} ...")
                rep -= 1
                continue
            print(f"[Fine-tune] Skipping: {e}")
            return model
    if X_raw.shape[0] < max(1, min_train_windows):
        print(f"[Fine-tune] Skipping: only {X_raw.shape[0]} window(s) available (need ≥{min_train_windows}).")
        return model
    m_out = len(PRODUCTS); H = y_raw.shape[1] // m_out
    X = (X_raw - x_scaler.mean_) / x_scaler.scale_
    if y_scaler is not None:
        y_mat = y_raw.reshape(-1, H, m_out).reshape(-1, m_out)
        y = ((y_mat - y_scaler.mean_) / y_scaler.scale_).reshape(-1, H*m_out)
    else:
        y = y_raw
    eff_bs = max(1, min(batch_size, X.shape[0]))
    model.fit(X, y, epochs=epochs, batch_size=eff_bs, verbose=0)
    print(f"[Fine-tune] Done on scenarios={scenarios} with repeat={rep}, windows={X.shape[0]}, epochs={epochs}")
    return model

# -------------------------------------------------------
# Forecast with the SAME feature pipeline used in training
# -------------------------------------------------------
def forecast_baseline_for_scenario(df, model, x_scaler, y_scaler, scen,
                                   horizon=HORIZON, lookback=LOOKBACK):
    """
    Rebuild the last window with make_windows_from_tidy for this scenario,
    scale with x_scaler, predict H×m_out, invert y_scaler, return DataFrame.

    NOTE: Horizon is effectively limited by the model's trained H
          (H_trained = output_dim / len(PRODUCTS)).
    """
    df_use = make_df_use(df)
    df_s   = df_use[df_use["scenario"] == scen].copy()

    # Build windows only to grab the *last* input window; horizon=1 is enough for that
    X_raw, _ = make_windows_from_tidy(df_s, lookback=lookback, horizon=1, augment_repeat=None)
    if X_raw.ndim != 3 or X_raw.shape[0] == 0:
        raise ValueError(f"make_windows_from_tidy produced no window for scenario={scen}")

    X_last = X_raw[-1]  # (L, m_in)

    # Sanity: ensure m_in matches scaler
    assert X_last.shape[1] == len(x_scaler.mean_), \
        f"m_in mismatch: {X_last.shape[1]} vs {len(x_scaler.mean_)}"

    # Scale features per-column
    X_in = (X_last - x_scaler.mean_[None, :]) / x_scaler.scale_[None, :]
    X_in = X_in[None, ...]  # (1, L, m_in)

    # Raw model output: flat vector of length H_trained * m_out
    yhat_flat = model.predict(X_in, verbose=0).reshape(-1)
    m_out = len(PRODUCTS)
    H_trained = yhat_flat.size // m_out

    if yhat_flat.size % m_out != 0:
        raise ValueError(f"Model output size {yhat_flat.size} is not divisible by m_out={m_out}.")

    # Use the *minimum* of requested horizon and model's trained horizon
    H_eff = min(horizon, H_trained)

    yhat_scaled = yhat_flat.reshape(H_trained, m_out)[:H_eff, :]  # (H_eff, m_out)

    if y_scaler is not None:
        yhat = (
            yhat_scaled * y_scaler.scale_.reshape(1, m_out) +
            y_scaler.mean_.reshape(1, m_out)
        )
    else:
        yhat = yhat_scaled

    out = pd.DataFrame(yhat, columns=PRODUCTS)
    out.insert(0, "horizon", np.arange(1, H_eff + 1))
    out.insert(0, "scenario", scen)
    return out


# -------------------------------------------------------
# Tariff / elasticity post-processing (optional)
# -------------------------------------------------------
def apply_tariff_elasticity(fore_df, tariff_path, elasticity, pass_through, china_share):
    adj = fore_df.copy()
    for idx, row in adj.iterrows():
        h = int(row["horizon"])
        tau = float(tariff_path.get(h, 0.0))
        for prod in PRODUCTS:
            eps = elasticity[prod]
            rho = pass_through[prod]
            sCN = china_share[prod]
            factor = 1.0 + eps * rho * tau * sCN
            adj.at[idx, prod] = max(0.0, row[prod] * factor)
    return adj


In [96]:
# -------------------------------------------------------
# Example usage
# -------------------------------------------------------
if __name__ == "__main__":
    df_raw = pd.read_csv(r"C:\Users\marieska\Downloads\monthly_demand_regions_weighted_2020_2024_growth_corrected_Adjusted.csv")
    df_use = make_df_use(df_raw)
    PRODUCTS = sorted({_norm_prod(p) for p in df_use["product_name"].unique()})
    print("PRODUCTS (normalized):", PRODUCTS)

    # 2) Train model zoo and pick best on validation sMAPE
    model, best_kind, x_sc, y_sc, L, m_in, H, table = run_model_zoo(df_use, LOOKBACK, HORIZON, augment_repeat=1)
    print("Best DL model:", best_kind)
    print(table)

    # 3) (Optional) Fine-tune on tougher scenarios (robust to low-window scenarios)
    hard = ["Holiday_Q4_Surge", "Launch_Spike_Sep"]
    model = finetune_on_scenarios(
        model, df_use, hard,
        lookback=min(LOOKBACK, 12),
        horizon=HORIZON,
        x_scaler=x_sc, y_scaler=y_sc,
        epochs=3, batch_size=32,
        augment_repeat=1
    )

    # 4) Forecast for baseline
    SCEN = "Baseline_Seasonal"
    fc = forecast_baseline_for_scenario(df_use, model, x_sc, y_sc, SCEN, horizon=HORIZON, lookback=LOOKBACK)
    print(fc.head(HORIZON))

    # 5) (Optional) tariff adjustment
    tariff_path = {1: 0.05, 2: 0.05, 3: 0.10}  # example path
    ELASTICITY   = {"iPhone": -1.2, "iPad": -0.8, "MacBook": -1.0}
    PASS_THROUGH = {"iPhone": 0.7,  "iPad": 0.6,  "MacBook": 0.65}
    CHINA_SHARE  = {"iPhone": 0.6,  "iPad": 0.5,  "MacBook": 0.55}
    fc_tariff = apply_tariff_elasticity(fc, tariff_path, ELASTICITY, PASS_THROUGH, CHINA_SHARE)
    print(fc_tariff.head(HORIZON))


PRODUCTS (normalized): ['MacBook', 'iPad', 'iPhone']
Best DL model: rnn
       kind      val_rmse  val_smape
0       rnn   4032.295493   4.199388
4  cnn_lstm  10231.835729   7.815067
3   cnn_rnn   9887.872111   8.101153
1      lstm  11090.118658   8.468427
2       cnn  11214.696970   8.670750
[Fine-tune] Done on scenarios=['Holiday_Q4_Surge', 'Launch_Spike_Sep'] with repeat=1, windows=74, epochs=3
             scenario  horizon       MacBook          iPad         iPhone
0   Baseline_Seasonal        1  39437.919757  36310.000013  133745.051780
1   Baseline_Seasonal        2  33363.900929  33872.749781  107601.750306
2   Baseline_Seasonal        3  32740.717801  34099.994696  106623.826766
3   Baseline_Seasonal        4  35027.286706  35515.332453  114329.740940
4   Baseline_Seasonal        5  36550.484220  36571.889321  117162.717638
5   Baseline_Seasonal        6  37951.259113  39211.896026  118704.736349
6   Baseline_Seasonal        7  39079.707425  39328.012998  120476.899906
7   Bas

In [97]:
df_use.head()

,Year,period,product_name,scenario,demand_units
0,2020,1,MacBook,Back_to_School,29434.0
1,2020,1,MacBook,Baseline_Seasonal,31116.0
2,2020,1,MacBook,Combined_Stress,31793.0
3,2020,1,MacBook,Holiday_Q4_Surge,30143.0
4,2020,1,MacBook,Launch_Spike_Sep,30322.0


In [98]:
# 1) Compare medians to last-year medians (scale sanity)
pv = panel_for_scenario_full(df_use, "Baseline_Seasonal")
print("last-12 medians:", pv.tail(12)[["iPhone","iPad","MacBook"]].median())
print("DL h=1–12 median:", fc[["iPhone","iPad","MacBook"]].median())

# 2) Compare DL to naive same-month-last-year on last H months
y_true_lastH = pv.values.astype("float64")[-HORIZON:, :]
yhat_naive   = naive_same_month_last_year(df_use, "Baseline_Seasonal", HORIZON, LOOKBACK)
print("DL vs last-H sMAPE (proxy backtest):", smape_vec(y_true_lastH, fc[["iPhone","iPad","MacBook"]].to_numpy()))
print("Naive vs last-H sMAPE:", smape_vec(y_true_lastH, yhat_naive))


last-12 medians: product_name
iPhone     117960.629750
iPad        38240.374160
MacBook     36386.311885
dtype: float64
DL h=1–12 median: iPhone     117933.726993
iPad        37891.892673
MacBook     37759.250063
dtype: float64
DL vs last-H sMAPE (proxy backtest): 5.566627335522847
Naive vs last-H sMAPE: 71.90267054766349


In [99]:
# ================================
# EVALUATION: last-fold comparison
# ================================
# --- metrics (epsilon-safe) ---

def _panel_for_scenario_rect(df, scen, products):
    d = make_df_use(df)
    d = normalize_time_strict(d)
    g = d[d["scenario"] == scen].copy()
    pv = (g.pivot_table(index="t_month", columns="product_name", values="demand_units", aggfunc="first")
            .reindex(columns=products))
    return d, pv

def _cut_df_to_t_end(df_use_norm, scen, t_end):
    """Return a df with only rows of the given scenario where t_month < t_end (drop helper col)."""
    cols_keep = [c for c in df_use_norm.columns if c != "t_month"]
    mask = (df_use_norm["scenario"]==scen) & (df_use_norm["t_month"] < t_end)
    return df_use_norm.loc[mask, cols_keep].copy()


def _to_array(df_fc, products=None):
    """H x m array from a forecast DataFrame with columns ['scenario','horizon', <products>]."""
    if products is None:
        products = [c for c in df_fc.columns if c not in ("scenario","horizon")]
    arr = df_fc.sort_values("horizon")[products].to_numpy(dtype="float64")
    return arr

# def smape_eps(y_true, y_pred, eps=1e-8):
#     """sMAPE (%) with epsilon to avoid 0/0 on small values."""
#     yt = np.asarray(y_true, dtype="float64").ravel()
#     yp = np.asarray(y_pred, dtype="float64").ravel()
#     denom = (np.abs(yt) + np.abs(yp)) / 2.0
#     return float(np.mean(np.abs(yt - yp) / np.maximum(denom, eps)) * 100.0)


# ---------- Metrics (drop-in patch) ----------
def smape_eps(y_true, y_pred, eps=1e-8):
    """
    Symmetric MAPE with numerical safeguard.
    Returns percent (0..200).
    """
    y_true = np.asarray(y_true, dtype="float64")
    y_pred = np.asarray(y_pred, dtype="float64")
    denom = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    return float(np.mean(np.abs(y_true - y_pred) / np.maximum(denom, eps)) * 100.0)

def wape_eps(y_true, y_pred, eps=1e-8):
    """
    Weighted absolute percentage error with safeguard.
    Returns percent (0..∞).
    """
    y_true = np.asarray(y_true, dtype="float64")
    y_pred = np.asarray(y_pred, dtype="float64")
    num = np.sum(np.abs(y_true - y_pred))
    den = np.sum(np.abs(y_true))
    return float((num / max(den, eps)) * 100.0)

def rmse_vec(y_true, y_pred):
    y_true = np.asarray(y_true, dtype="float64")
    y_pred = np.asarray(y_pred, dtype="float64")
    return float(np.sqrt(np.mean((y_true - y_pred) ** 2)))

def rmsle_log1p(y_true, y_pred, eps=1e-12):
    y_true = np.asarray(y_true, dtype="float64")
    y_pred = np.asarray(y_pred, dtype="float64")
    return float(np.sqrt(np.mean((np.log1p(np.maximum(y_true, 0)+eps) -
                                  np.log1p(np.maximum(y_pred, 0)+eps)) ** 2)))

def wape_eps(y_true, y_pred, eps=1e-8):
    """WAPE (%) with epsilon on denominator."""
    yt = np.asarray(y_true, dtype="float64").ravel()
    yp = np.asarray(y_pred, dtype="float64").ravel()
    return float(np.sum(np.abs(yt - yp)) / np.maximum(np.sum(np.abs(yt)), eps) * 100.0)

def rmse_vec(y_true, y_pred):
    yt = np.asarray(y_true, dtype="float64").ravel()
    yp = np.asarray(y_pred, dtype="float64").ravel()
    return float(np.sqrt(np.mean((yt - yp)**2)))

def rmsle_log1p(y_true, y_pred, clip_min=0.0):
    yt = np.asarray(y_true, dtype="float64").ravel()
    yp = np.asarray(y_pred, dtype="float64").ravel()
    yp = np.maximum(yp, clip_min)  # avoid log of negatives
    return float(np.sqrt(np.mean((np.log1p(yt) - np.log1p(yp))**2)))

####------------------------------------------------------
# ---------- DL forecast helper that matches the training pipeline ----------
def _make_extras_for_last_L(df_norm, scen, lookback, products, expected_m_in):
    """
    Build the SAME extras used during DL training:
      - 11 month one-hots m_2..m_12 (no m_1)
      - normalized trend t_norm in [0,1]
      - Fourier seasonality (sin/cos) based on month
    Returns an (L, n_extras) float32 array.
    """
    # We assume df_norm already has: scenario, Year, period, t_month, product_name, demand_units
    g = (df_norm[df_norm["scenario"] == scen]
         .drop_duplicates(subset=["t_month"])
         .sort_values("t_month")[["t_month", "Year", "period"]])

    if len(g) < lookback:
        raise ValueError(f"Not enough rows for lookback={lookback}. Have {len(g)}.")

    last = g.iloc[-lookback:].copy()
    # Month one-hots m_2..m_12 (no m_1)
    for k in range(2, 13):
        last[f"m_{k}"] = (last["period"].astype(int) == k).astype(float)

    # Trend normalization on t_month (within the entire scenario time span)
    t_min, t_max = g["t_month"].min(), g["t_month"].max()
    denom = max(1.0, float(t_max - t_min))
    last["t_norm"] = (last["t_month"] - t_min) / denom

    # Fourier seasonality from month
    m = last["period"].astype(int).to_numpy()
    last["month_sin"] = np.sin(2*np.pi*(m/12.0))
    last["month_cos"] = np.cos(2*np.pi*(m/12.0))

    # Final extras in the same canonical order we’ll assume was used in training:
    extra_cols = [f"m_{k}" for k in range(2,13)] + ["t_norm", "month_sin", "month_cos"]
    X_extras = last[extra_cols].to_numpy(dtype="float32")

    # Sanity: check width: 11 + 1 + 2 = 14
    # Products are 3 → total expected 17
    total_expected = len(products) + X_extras.shape[1]
    if total_expected != expected_m_in:
        raise AssertionError(
            f"Extras width mismatch. Products={len(products)}, extras={X_extras.shape[1]} "
            f"→ total {total_expected} but model expects m_in={expected_m_in}. "
            f"Adjust extras to match training."
        )
    return X_extras


def forecast_dl_for_eval(model, x_scaler, y_scaler, df_use,
                         scen, products, lookback, horizon,
                         m_in_trained, m_out_trained):
    """
    Build an input of shape (1, L, m_in_trained) that matches training:
    [PRODUCTS (3)] + [m_2..m_12 (11)] + [t_norm (1)] + [month_sin, month_cos (2)] = 17
    """
    # 1) Continuous timeline + panel (T × m_out)
    df_norm = normalize_time_strict(df_use)
    pv = panel_for_scenario_matrix_safe(
        df_norm, scen,
        prod_col="product_name", dem_col="demand_units",
        time_col="t_month", scen_col="scenario",
        products=products, impute="drop"
    )  # index=t_month, columns=PRODUCTS

    # 2) Last L window of the 3 product series
    last_L_products = pv.values.astype("float32")[-lookback:, :]   # (L, 3)

    # 3) Recreate the extras exactly as in training
    X_extras = _make_extras_for_last_L(df_norm, scen, lookback, products, expected_m_in=m_in_trained)  # (L, 14)

    # 4) Concatenate to width 17
    last_L_full = np.concatenate([last_L_products, X_extras], axis=1)  # (L, 17)
    assert last_L_full.shape[1] == m_in_trained

    # 5) Scale with TRAIN stats, predict, invert y scale
    X_in = (last_L_full - x_scaler.mean_) / x_scaler.scale_
    X_in = X_in[None, ...]  # (1, L, 17)

    yhat_scaled = model.predict(X_in, verbose=0).reshape(horizon, m_out_trained)
    if y_scaler is not None:
        yhat = yhat_scaled * y_scaler.scale_.reshape(1, m_out_trained) + y_scaler.mean_.reshape(1, m_out_trained)
    else:
        yhat = yhat_scaled
    yhat = np.clip(yhat, 0.0, None)
    return yhat


#---------------------------------------------------------------------------------------

  
def evaluate_last_fold(
    df_raw,                                    # original dataframe
    scen="Baseline_Seasonal",
    model_name_map=None,                       # dict: {name: forecast_df} optional (XGB/Prophet/SARIMAX/Ensembles)
    dl_tuple=None,                             # (model, x_scaler, y_scaler) for your deep model
    products=None,
    lookback=None,
    horizon=None,
    save_prefix="eval_lastfold"
):
    """
    Cuts the scenario at the last feasible fold (t_end), forecasts the next H months per model,
    compares against ground truth, prints & saves tables.
    """
    products = products or PRODUCTS
    lookback = lookback or LOOKBACK
    horizon  = horizon  or HORIZON
    model_name_map = dict(model_name_map or {})  # copy

    # 1) set up normalized timeline & panel
    df_norm, panel = _panel_for_scenario_rect(df_raw, scen, products)
    mat_full = panel.to_numpy(float)
    T = mat_full.shape[0]
    if T < lookback + horizon + 1:
        raise ValueError(f"Not enough history T={T} for lookback={lookback}, horizon={horizon}")

    t_end = T - horizon  # last fold anchor
    y_true = mat_full[t_end : t_end + horizon, :]  # H × m

    print(f"\nEvaluating against last-fold (t_end={t_end}) ground truth for scenario '{scen}'.\n")
    rows_all, rows_pp = [], []
        # ---- Deep Learning model scoring ----
    if dl_tuple is not None:
        mdl, xs, ys, Ldl, mindl, Hdl = dl_tuple
        assert Hdl == horizon, f"DL horizon {Hdl} != eval horizon {horizon}"
        y_pred_dl = forecast_dl_for_eval(
            mdl, xs, ys, df_raw, scen, products,
            lookback=lookback, horizon=horizon,
            m_in_trained=mindl, m_out_trained=len(products)
        )
    
        rows_all.append({
            "model": "DL_Best",
            "N points": int(y_pred_dl.size),
            "sMAPE_eps (%)": smape_eps(y_true, y_pred_dl),
            "WAPE_eps (%)":  wape_eps(y_true, y_pred_dl),
            "RMSE":          rmse_vec(y_true, y_pred_dl),
            "RMSLE (log1p)": rmsle_log1p(y_true, y_pred_dl),
        })
    
        for j, prod in enumerate(products):
            rows_pp.append({
                "product_name": prod, "model": "DL_Best",
                "N points": int(horizon),
                "sMAPE_eps (%)": smape_eps(y_true[:, j], y_pred_dl[:, j]),
                "WAPE_eps (%)":  wape_eps(y_true[:, j], y_pred_dl[:, j]),
                "RMSE":          rmse_vec(y_true[:, j], y_pred_dl[:, j]),
                "RMSLE (log1p)": rmsle_log1p(y_true[:, j], y_pred_dl[:, j]),
            })


    
    overall = pd.DataFrame(rows_all).sort_values("sMAPE_eps (%)").reset_index(drop=True)
    perprod = pd.DataFrame(rows_pp).sort_values(["product_name","sMAPE_eps (%)"]).reset_index(drop=True)

    # 5) build a sMAPE matrix (rows=product, cols=model)
    sm_mat = perprod.pivot(index="product_name", columns="model", values="sMAPE_eps (%)")
    sm_mat = sm_mat.reindex(index=products)  # keep product order

    # 6) print nicely
    print("=== Overall (incl. ensembles/DL) ===")
    print(overall.to_string(index=False))
    print("\n=== Per-product (incl. ensembles/DL) ===")
    print(perprod.to_string(index=False))
    print("\n=== sMAPE_eps matrix (rows=product, cols=model) ===")
    print(sm_mat.round(2).to_string())

    # 7) save CSVs for Word
    overall.to_csv(f"{save_prefix}_overall.csv", index=False)
    perprod.to_csv(f"{save_prefix}_per_product.csv", index=False)
    sm_mat.to_csv(f"{save_prefix}_smape_matrix.csv")

    # 8) also return them in case you want to keep in memory
    return overall, perprod, sm_mat



# PART2: XGBoost, SARIMA,..... (Machine Learning Models) 

In [100]:
#BLOCK 1:CELL 1 — Imports & global config #
# Assumptions & tiny helpers
from sklearn.model_selection import TimeSeriesSplit
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
import numpy as np, pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings
import matplotlib.pyplot as plt
from prophet import Prophet

# --- CONFIG---
LOOKBACK = 12    # L: timesteps to look back (e.g., 12 months)
LOOKBACK_PLAN = 12
HORIZON  = 3     # H: steps ahead to predict (e.g., next 6 months)
HORIZON_PLAN=3
VAL_FRAC = 0.20  # fraction of data for validation (chronological)
# TEST_FRAC=0.15 # fraction for test
BATCH    = 32  
EPOCHS   = 300
PATIENCE = 30
LR       = 1e-3
SEED     = 42

PRODUCTS = ["iPhone","iPad","MacBook"]

ELASTICITY   = {"iPhone": -1.1, "iPad": -0.9, "MacBook": -1.3}
PASS_THROUGH = {"iPhone": 0.35, "iPad": 0.30, "MacBook": 0.40}
CHINA_SHARE  = {"iPhone": 0.65, "iPad": 0.55, "MacBook": 0.45}
TARIFF_PATH  = {h: 0.25 for h in range(1, HORIZON+1)}


df = pd.read_csv(r"C:\Users\marieska\Downloads\monthly_demand_regions_weighted_2020_2024_growth_corrected_Adjusted.csv")

In [101]:
#CELL 2 — Generic helpers (single source of truth)

def _pick_col(df, candidates, required=True):
    for c in candidates:
        if c in df.columns:
            return c
    if required:
        raise KeyError(f"None of the columns {candidates} found. Available: {list(df.columns)}")
    return None


##----------------------------------------------------------------------------------------------------------    

def ensure_time_columns_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Standardize column names & types that the rest of the pipeline expects.
    Guarantees the presence of: scenario, Year, period, product_name, demand_units
    If a 'ds' (timestamp) exists, derives Year/period from it.
    """
    df = df.copy()

    year_col = _pick_col(df, ["Year", "year"], required=False)
    per_col  = _pick_col(df, ["period", "month", "Month"], required=False)
    ds_col   = _pick_col(df, ["ds", "date", "Date"], required=False)

    if ds_col is not None and (year_col is None or per_col is None):
        ds = pd.to_datetime(df[ds_col])
        df["Year"]   = ds.dt.year.astype(int)
        df["period"] = ds.dt.month.astype(int)
        df["ds"]     = ds.dt.to_period("M").dt.to_timestamp(how="start")
    else:
        if year_col is None or per_col is None:
            raise KeyError("Need either 'ds' or both (Year/year) and (period/month/Month).")

        df["Year"]   = pd.to_numeric(df[year_col], errors="coerce").astype("Int64")
        df["period"] = pd.to_numeric(df[per_col],  errors="coerce").astype("Int64")

        # Validate 1..12
        if not df["period"].between(1, 12).fillna(False).all():
            bad = df.loc[~df["period"].between(1,12).fillna(False), ["Year","period"]]
            raise ValueError(f"'period' must be 1..12. Bad rows:\n{bad.head()}")

        df["ds"] = pd.to_datetime(dict(
            year=df["Year"].astype(int),
            month=df["period"].astype(int),
            day=1
        ))

    # Final types + sort
    df["Year"]   = df["Year"].astype(int)
    df["period"] = df["period"].astype(int)
    df = df.sort_values(["Year", "period"]).reset_index(drop=True)
    return df


##----------------------------------------------------------------------------------------------------------


def ensure_time_index(df):
    """
    Build a true month timeline per scenario and return:
    (DataFrame_with_t_month, scen_col, prod_col, year_col, per_col, dem_col)
    """
    d = df.copy()
    scen_col = _pick_col(d, ["scenario"])
    prod_col = _pick_col(d, ["product_name","product"])
    year_col = _pick_col(d, ["year","Year"], required=False)
    per_col  = _pick_col(d, ["period","month","Month"])
    dem_col  = _pick_col(d, ["demand_units","demand","qty","quantity"])

    d[per_col] = pd.to_numeric(d[per_col], errors="coerce").astype(int)

    if year_col is not None:
        d = d.sort_values([scen_col, year_col, per_col])
        d["ds"] = pd.to_datetime(dict(
            year=pd.to_numeric(d[year_col], errors="coerce").astype(int),
            month=d[per_col], day=1
        ))
    else:
        d = d.sort_values([scen_col, per_col])
        d["_yr_seq"] = d.groupby([scen_col, per_col]).cumcount()
        d["ds"] = pd.to_datetime(dict(year=2000 + d["_yr_seq"], month=d[per_col], day=1))

    d["t_month"] = d.groupby(scen_col)["ds"].rank(method="dense").astype(int) - 1
    return d, scen_col, prod_col, year_col, per_col, dem_col

#--------------------------------------------------------------------------------------------
def normalize_time_strict(d, sc="scenario", yc="Year", mc="period"):
    """Create a continuous month index per scenario."""
    d = d.copy()
    d[yc] = d[yc].astype(int)
    d[mc] = d[mc].astype(int)
    months = (d[[sc, yc, mc]].drop_duplicates()
              .sort_values([sc, yc, mc]).reset_index(drop=True))
    months["t_month"] = months.groupby(sc).cumcount()
    return d.merge(months, on=[sc, yc, mc], how="left")

def smape_vec(y_true, y_pred):  # percent
    y_true = np.asarray(y_true, float).ravel()
    y_pred = np.asarray(y_pred, float).ravel()
    denom = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    return float(np.mean(np.abs(y_true - y_pred) / np.maximum(denom, 1e-8)) * 100.0)

def mase_from_insample(y_insample, y_true, y_pred, m=12):
    """MASE per Hyndman & Koehler: needs INSAMPLE series for the seasonal naive denominator."""
    y_insample = np.asarray(y_insample, float).ravel()
    y_true     = np.asarray(y_true,     float).ravel()
    y_pred     = np.asarray(y_pred,     float).ravel()
    if len(y_insample) <= m: 
        return np.nan
    denom = np.mean(np.abs(y_insample[m:] - y_insample[:-m]))
    num   = np.mean(np.abs(y_true - y_pred))
    return float(num / np.maximum(denom, 1e-12))

    
def wape_percent(y_true, y_pred):  # percent
    y_true = np.asarray(y_true, float).ravel()
    y_pred = np.asarray(y_pred, float).ravel()
    return float(100.0 * np.sum(np.abs(y_true - y_pred)) /
                 np.maximum(np.sum(np.abs(y_true)), 1e-8))

def mase(y_true, y_pred, seasonal_period=12):
    """
    MASE = MAE(model) / MAE(naive seasonal, lag = seasonal_period).
    Returns np.nan if there aren't enough points for the seasonal naive.
    """
    y_true = np.asarray(y_true, dtype=float).ravel()
    y_pred = np.asarray(y_pred, dtype=float).ravel()
    mae = np.mean(np.abs(y_true - y_pred))
    if len(y_true) <= seasonal_period:
        return np.nan  # not enough data to compute seasonal naive
    naive_err = np.abs(y_true[seasonal_period:] - y_true[:-seasonal_period])
    denom = np.mean(naive_err)
    if denom < 1e-8:
        return np.inf  # or return 0.0 depending on your preference
    return float(mae / denom)

def wape(y_true, y_pred):
    y_true = np.asarray(y_true, dtype="float64")
    y_pred = np.asarray(y_pred, dtype="float64")
    return float(np.sum(np.abs(y_true - y_pred)) / (np.sum(np.abs(y_true)) + 1e-8) * 100.0)

def rmse_vec(y_true, y_pred):
    y_true = np.asarray(y_true, dtype="float64")
    y_pred = np.asarray(y_pred, dtype="float64")
    return float(np.sqrt(np.mean((y_true - y_pred)**2)))



##---------------------- here would be panel_for_scenario (region aggregation)  and panel_for_scenario_matrix_safe (your “safe” pivot)
def panel_for_scenario(df, scen, products=PRODUCTS):
    g = df.loc[df["scenario"] == scen].copy()
    if "region" in g.columns:
        g = g.groupby(["period", "product_name"], as_index=False)["demand_units"].sum()

    pv = (g.assign(product_name=lambda x: x["product_name"].astype(str).str.strip())
            .pivot(index="period", columns="product_name", values="demand_units")
            .reindex(columns=products)
            .sort_index())

    if pv.isna().any().any():
        raise ValueError(f"NaNs after pivot for scenario '{scen}'.")
    return pv


def panel_for_scenario_matrix(df, scen, prod_col, dem_col, impute="drop"):
    g = df[df["scenario"] == scen].copy()
    pt = (g.pivot_table(index="t_month", columns=prod_col, values=dem_col, aggfunc="first")
            .reindex(columns=PRODUCTS))
    if impute == "drop":
        pt = pt.dropna(axis=0, how="any")
    elif impute == "zero":
        pt = pt.fillna(0.0)
    elif impute == "ffill":
        pt = pt.sort_index().ffill().dropna(axis=0, how="any")
    else:
        raise ValueError("impute must be 'drop', 'zero', or 'ffill'.")
    if pt.empty:
        raise ValueError(f"No complete months for scenario {scen} after imputation='{impute}'.")
    return pt.to_numpy(dtype="float64")

# ===============================
# 3) Safe panel builder (T × m)
# ===============================


def panel_for_scenario_matrix_safe(df, scen, prod_col, dem_col,
                                   time_col="t_month", scen_col="scenario",
                                   products=None, impute="drop"):
    # 1) checks
    if not hasattr(df, "columns"):
        raise TypeError(f"'df' must be a pandas DataFrame, got: {type(df)}")
    for col in (scen_col, prod_col, dem_col, time_col):
        if col not in df.columns:
            raise KeyError(f"Column '{col}' not in df.columns.")
    if products is None:
        products = list(df[prod_col].drop_duplicates())

    # 2) filter + pivot
    g = df.loc[df[scen_col] == scen, [time_col, prod_col, dem_col]].copy()
    pt = (g.pivot_table(index=time_col, columns=prod_col,
                        values=dem_col, aggfunc="first")
            .reindex(columns=products))

    # 3) impute
    if impute == "drop":
        pt = pt.dropna()
    elif impute == "ffill":
        pt = pt.sort_index().ffill()
    elif impute == "zero":
        pt = pt.fillna(0.0)
    else:
        raise ValueError("impute must be 'drop' | 'ffill' | 'zero'")

    return pt



# ===============================
# 1) Canonical dataset builder
# ===============================
def make_df_use(df):
    """Aggregate (if region exists), ensure a scenario column,
    and standardize column names expected by the pipeline."""
    Y  = "Year" if "Year" in df.columns else "year"
    P  = "period" if "period" in df.columns else ("month" if "month" in df.columns else "Month")
    PN = "product_name" if "product_name" in df.columns else ("product" if "product" in df.columns else "product_id")
    V  = ("demand_units" if "demand_units" in df.columns else
          ("demand" if "demand" in df.columns else
           ("qty" if "qty" in df.columns else "quantity")))
    d = df.copy()

    if "region" in d.columns:
        d = d.groupby([Y, P, PN], as_index=False)[V].sum()

    if "scenario" not in d.columns:
        d["scenario"] = "Baseline_Seasonal"

    d = d.rename(columns={Y:"Year", P:"period", PN:"product_name", V:"demand_units"})
    d["Year"]   = d["Year"].astype(int)
    d["period"] = d["period"].astype(int)
    return d




def add_calendar_cols(df: pd.DataFrame) -> pd.DataFrame:
    
    df = ensure_time_columns(df).copy()  # guarantees Year/period/ds
    df["date"]  = df["ds"]
    df["month"] = df["period"].astype(int)
    df["year"]  = df["Year"].astype(int)

    # Seasonal encodings
    df["m_sin"] = np.sin(2*np.pi*df["month"]/12.0)
    df["m_cos"] = np.cos(2*np.pi*df["month"]/12.0)
    return df

#---------------------------------------------------------------------------------------------------------


In [102]:
###### CELL 3 — Canonical data + the 6 consistency steps (must run before any model)

df_use = make_df_use(df)                 # your function
df_use = ensure_time_columns_df(df_use)  # guarantees Year, period, ds

df2 = normalize_time_strict(df_use)      # adds t_month
SCEN = "Baseline_Seasonal"
panel = panel_for_scenario_matrix_safe(
    df2, SCEN,
    prod_col="product_name", dem_col="demand_units",
    time_col="t_month", scen_col="scenario",
    products=PRODUCTS, impute="drop"
)
print("Panel shape:", panel.shape, "Any NaN?", panel.isna().any().any())

pivot_raw = (df2[df2["scenario"]==SCEN]
             .pivot_table(index="t_month", columns="product_name", values="demand_units", aggfunc="first")
             .reindex(columns=PRODUCTS))
print("Missing per product (pre-impute):", pivot_raw.isna().sum().to_dict())



mat_full = panel.to_numpy("float64")
T = mat_full.shape[0]
max_start = T - HORIZON
anchors = list(range(LOOKBACK, max_start, max(1, (max_start-LOOKBACK)//4)))[-4:]
t_end = anchors[-1]



Panel shape: (60, 3) Any NaN? False
Missing per product (pre-impute): {'iPhone': 0, 'iPad': 0, 'MacBook': 0}


In [103]:
##CELL4
# =========================
# XGBOOST: tabular dataset + trainer + predictor
# =========================
from collections import defaultdict

try:
    from xgboost import XGBRegressor
except ImportError as e:
    raise ImportError("Please install xgboost: pip install xgboost") from e

def _month_one_hot(m):  # m in 1..12
    oh = np.zeros(12, dtype=np.float32)
    oh[int(m)-1] = 1.0
    return oh

def _flags_for_month(m):
    # simple example flags to mirror your DL exogenous inputs
    flag_launch  = 1.0 if int(m) == 9 else 0.0
    flag_holiday = 1.0 if int(m) in (10,11,12) else 0.0
    return np.array([flag_launch, flag_holiday], dtype=np.float32)

def _panel_products(df, scen, products):
    """Return (period_seq, matrix T×m) for a scenario."""
    g = (df[df["scenario"] == scen]
           .groupby(["Year","period","product_name"], as_index=False)["demand_units"]
           .sum()
           .sort_values(["Year","period"]))
    # we need the chronological month sequence (1..12 repeating)
    period_seq = g[["Year","period"]].drop_duplicates().sort_values(["Year","period"])["period"].to_numpy()
    pv = (g.pivot(index=["Year","period"], columns="product_name", values="demand_units")
            .sort_index()
            .reset_index(drop=True)
            .reindex(columns=products))
    M = pv.to_numpy(dtype=np.float32)
    # fill any gaps with zeros (or ffill if you prefer)
    M = np.nan_to_num(M, nan=0.0)
    return period_seq.astype(int), M

def build_tabular_dataset(df, lookback, horizon, products=PRODUCTS):
    """
    Build rolling windows across *all scenarios*.
    X features = flattened last L×m product block  +  one-hot(month_target) + flags(target_month)
    y targets  = next h step for each product (we train one regressor per (product, h))
    Returns: dict with
        feat_windows: (N, L*m)    # common for all h
        target_months: (N, horizon) ints in 1..12 for each sample & horizon
        ys: dict[(prod_idx, h)] -> (N,)
        meta: {'L':..., 'm':..., 'horizon':..., 'N':...}
    """
    dfu = make_df_use(df)
    scenarios = dfu["scenario"].unique().tolist()

    feat_list = []                         # flattened L×m window
    target_months = []                     # (N, horizon)
    ys = defaultdict(list)                 # per (j,h) target

    for scen in scenarios:
        period_seq, M = _panel_products(dfu, scen, products)  # T×m
        T, m = M.shape
        if T < lookback + horizon:  # not enough for this scenario
            continue

        for start in range(0, T - lookback - horizon + 1):
            end = start + lookback
            block = M[start:end, :]                            # (L, m)
            feat_list.append(block.reshape(-1))                # (L*m,)

            # compute target months for each horizon step
            months_h = []
            for h in range(1, horizon+1):
                tgt_idx = end + (h - 1)
                months_h.append(int(period_seq[tgt_idx]))
                # push targets per product
                for j in range(m):
                    ys[(j, h)].append(float(M[tgt_idx, j]))
            target_months.append(months_h)

    if len(feat_list) == 0:
        raise ValueError("No windows built. Check LOOKBACK/HORIZON versus data length.")

    X_base = np.asarray(feat_list, dtype=np.float32)          # (N, L*m)
    target_months = np.asarray(target_months, dtype=int)      # (N, H)
    N = X_base.shape[0]
    L = lookback
    m = len(products)

    meta = {"L": L, "m": m, "horizon": horizon, "N": N}

    # Convert dict of lists -> arrays
    ys_arr = {k: np.asarray(v, dtype=np.float32) for k, v in ys.items()}

    return X_base, target_months, ys_arr, meta

def _build_features_for_h(X_base, target_months, h):
    """
    Concatenate base window with exogenous for horizon step h:
    X_h = [X_base, one_hot(month_h), flags(month_h)]
    """
    N = X_base.shape[0]
    months_h = target_months[:, h-1]                   # (N,)
    oh = np.vstack([_month_one_hot(m) for m in months_h])      # (N,12)
    fl = np.vstack([_flags_for_month(m) for m in months_h])    # (N,2)
    return np.concatenate([X_base, oh, fl], axis=1)            # (N, L*m + 14)

def train_xgb_forecasters(df, lookback, horizon, products=PRODUCTS):
    """
    Trains one XGBRegressor per (product, horizon_step).
    Uses a chronological 80/20 split over the constructed windows for quick validation.
    Returns:
      models: dict[(product_name, h)] -> XGBRegressor
      quick_smape: dict[product_name] -> mean sMAPE over h on the validation slice
    """
    try:
        from xgboost import XGBRegressor
    except ImportError:
        raise ImportError("Install xgboost: pip install xgboost")

    # build dataset
    X_base, target_months, ys_arr, meta = build_tabular_dataset(df, lookback, horizon, products)
    N = meta["N"]
    n_val = max(1, int(round(N * 0.2)))
    n_tr  = N - n_val

    models = {}
    smape_per_prod = defaultdict(list)

    # Use normalized products (and what’s really present in ys_arr)
    prods_train = sorted({_norm_prod(p) for p in products})
    # In some pipelines ys_arr is keyed by (prod,h) with prod’s original spelling; normalize a view:
    # (If your ys_arr is already accessed by index only, this is benign)
    # We will just rely on the 'products' list we passed to build_tabular_dataset.

    for j, prod in enumerate(products):
        prod_n = _norm_prod(prod)

        for h in range(1, horizon+1):
            X_h = _build_features_for_h(X_base, target_months, h)   # (N, L*m + 14)
            y   = ys_arr[(j, h)]                                    # (N,)

            Xtr, ytr = X_h[:n_tr], y[:n_tr]
            Xvl, yvl = X_h[n_tr:],  y[n_tr:]

            model = XGBRegressor(
                n_estimators=400,
                max_depth=4,
                learning_rate=0.05,
                subsample=0.9,
                colsample_bytree=0.9,
                reg_lambda=1.0,
                objective="reg:squarederror",
                random_state=SEED,
            )
            model.fit(Xtr, ytr, eval_set=[(Xvl, yvl)], verbose=False)
            models[(prod_n, h)] = model  # <-- store under NORMALIZED product key

            if len(yvl) > 0:
                yhat_vl = model.predict(Xvl)
                sm = smape_vec(yvl, yhat_vl)
                smape_per_prod[prod_n].append(sm)

    quick_smape = {prod_n: float(np.mean(v)) if len(v) else np.nan
                   for prod_n, v in smape_per_prod.items()}

    print("XGB models trained for products (normalized):", sorted({k[0] for k in models.keys()}))
    return models, quick_smape


def forecast_xgb(models, df, scen, lookback, horizon, products=PRODUCTS):
    """
    Make an H×m forecast for a scenario using trained per-(product,h) models.
    Uses the *last* lookback window from the scenario and rolls months forward.
    """
    dfu = make_df_use(df).copy()
    # normalize product names in the DF so pivot/selection aligns with model keys
    dfu["product_name"] = dfu["product_name"].astype(str).str.strip()

    # normalize the requested products order
    products_n = [_norm_prod(p) for p in products]

    # get last window block (L×m) over the chosen scenario
    period_seq, M = _panel_products(dfu, scen, products_n)  # T×m (columns in products_n order)
    T, m = M.shape
    if T < lookback:
        raise ValueError(f"Not enough history for scenario={scen}: T={T} < L={lookback}")

    # base features (flattened last L×m)
    block = M[-lookback:, :]                      # (L, m)
    base = block.reshape(-1).astype(np.float32)   # (L*m,)

    last_month = int(period_seq[-1])              # 1..12
    rows = []
    for h in range(1, horizon+1):
        # roll calendar forward
        target_month = ((last_month - 1 + h) % 12) + 1
        feat = np.concatenate(
            [base, _month_one_hot(target_month), _flags_for_month(target_month)],
            axis=0
        )[None, :]

        preds = []
        for j, prod in enumerate(products_n):
            # try normalized key first
            mdl = models.get((prod, h))
            if mdl is None:
                # (optional) backward-compat: if someone trained with original key
                mdl = models.get((_norm_prod(prod), h))
            if mdl is None:
                raise KeyError(
                    f"XGB model not found for product '{prod}' at horizon h={h}. "
                    f"Available keys sample: {list(models.keys())[:6]}..."
                )
            preds.append(float(mdl.predict(feat)[0]))
        rows.append(preds)

    out = pd.DataFrame(rows, columns=products_n)
    out.insert(0, "horizon", np.arange(1, horizon+1))
    out.insert(0, "scenario", scen)
    return out

# Convenience wrappers used in your code
def train_xgb(df_train):
    return train_xgb_forecasters(df_train, LOOKBACK, HORIZON)[0]

def predict_xgb_df(models_dict, df_train, scen, lookback, horizon):
    return forecast_xgb(models_dict, df_train, scen, lookback, horizon)



In [104]:
# #CELL 5 — SARIMAX (with exog, fixed columns) 
# # ---- SARIMAX (fixed exog dtype + stable columns) ----

def _unpack_sarimax_record(rec):
    """
    Accepts a SARIMAX model record in either format:
      • dict: {"res": <results>, "log1p": bool}
      • tuple/list: (<results>, <optional bool log1p>)
    Returns (res, log1p_bool).
    """
    import numpy as np

    # dict case
    if isinstance(rec, dict):
        res = rec.get("res")
        if res is None:
            raise KeyError("SARIMAX record dict missing 'res'.")
        return res, bool(rec.get("log1p", False))

    # tuple/list case
    if isinstance(rec, (tuple, list)):
        if len(rec) == 2:
            return rec[0], bool(rec[1])
        elif len(rec) >= 1:
            # if only the results are stored, assume no log1p
            return rec[0], False

    raise TypeError(f"Unrecognized SARIMAX record type: {type(rec)} with value: {rec}")


def sarimax_forecast_df(models, df, scen, horizon):
    """
    Build an H×m forecast DataFrame from stored SARIMAX fits.
    Works whether models[(scen, prod)] is a dict or a tuple.
    """
    import numpy as np
    rows = []
    used_key_shape = None

    for prod in PRODUCTS:
        # Prefer (scenario, product) key; fall back to product-only if needed
        if (scen, prod) in models:
            rec = models[(scen, prod)]
            used_key_shape = "(scenario, product)"
        elif prod in models:
            rec = models[prod]
            used_key_shape = "(product)"
        else:
            raise KeyError(f"SARIMAX model not found for product '{prod}' (scenario='{scen}').")

        res, log1p = _unpack_sarimax_record(rec)

        # Forecast next horizon steps
        fc = res.get_forecast(steps=horizon).predicted_mean  # pandas Series or array
        yhat = np.asarray(fc, dtype="float64").reshape(-1)

        # invert log1p if necessary, clip negatives
        if log1p:
            yhat = np.expm1(yhat)
        yhat = np.clip(yhat, 0.0, None)

        rows.append(yhat)

    # stack to H×m and return DataFrame
    arr = np.vstack(rows).T  # shape (H, m) after stacking per-product rows
    out = pd.DataFrame(arr, columns=PRODUCTS)
    out.insert(0, "horizon", np.arange(1, horizon + 1))
    out.insert(0, "scenario", scen)
    return out


def predict_sarimax_df(model_obj, df_train, scen, lookback, horizon):
    # Pass-through wrapper to keep your call sites unchanged
    return sarimax_forecast_df(model_obj, df_train, scen=scen, horizon=horizon)


In [105]:
def _ensure_canonical(df):
    # Your normalizer from earlier
    return ensure_time_columns_df(df)

def _panel_for_plot(df, scen, products):
    d = df.copy()
    if "region" in d.columns:
        d = (d.groupby(["scenario", "Year", "period", "product_name"], as_index=False)
               ["demand_units"].sum())
    d = d[d["scenario"] == scen].copy()
    d = d.sort_values(["Year", "period", "product_name"])
    d["t_month"] = (d["Year"] - d["Year"].min())*12 + (d["period"] - 1)
    pt = (d.pivot_table(index="t_month", columns="product_name",
                        values="demand_units", aggfunc="first")
            .reindex(columns=products))
    return pt, d

def plot_sarimax_forecast(models, df, scen, products, horizon=12, title=None):
    df = _ensure_canonical(df)
    pt, d = _panel_for_plot(df, scen, products)

    # Build future month dummies (m_2..m_12) for the next H steps
    last_period = int(d.loc[d["scenario"]==scen, "period"].max())
    fut_months = ((np.arange(1, horizon+1) + last_period - 1) % 12) + 1
    exog_cols = [f"m_{m}" for m in range(2,13)]
    fut_cat = pd.Categorical(fut_months, categories=np.arange(1,13), ordered=True)
    ex_fut_df = pd.get_dummies(fut_cat, drop_first=True).astype("float64")
    ex_fut_df.columns = exog_cols
    ex_fut_df = ex_fut_df.reindex(columns=exog_cols, fill_value=0.0)
    ex_fut = ex_fut_df.values  # (H, 11)

    nrows = len(products)
    fig, axes = plt.subplots(nrows, 1, figsize=(11, 2.6*nrows), sharex=True)
    if nrows == 1:
        axes = [axes]

    x_hist = pt.index.values
    for ax, prod in zip(axes, products):
        series = pt[prod].astype("float64").values
        ax.plot(x_hist, series, lw=2, label="Actual")

        info = models[(scen, prod)]
        res  = info["res"]
        # Forecast with aligned exogenous
        fc    = res.get_forecast(steps=horizon, exog=ex_fut)
        mean  = fc.predicted_mean
        ci    = fc.conf_int(alpha=0.05)  # DataFrame with 2 cols

        # If log1p was used, invert
        if info.get("log1p", True):
            mean = np.expm1(mean)
            ci   = np.expm1(ci)

        x_fc = np.arange(x_hist[-1] + 1, x_hist[-1] + 1 + horizon)
        ax.plot(x_fc, mean.values, lw=2, linestyle="--", label="Forecast")
        ax.fill_between(x_fc, ci.iloc[:,0].values, ci.iloc[:,1].values,
                        alpha=0.2, label="95% CI")

        ax.set_title(f"{prod} – {scen}")
        ax.set_ylabel("Demand (units)")
        ax.grid(True, alpha=0.3)

    axes[-1].set_xlabel("t_month (monotone index)")
    if title:
        fig.suptitle(title, y=0.98)
    handles, labels = axes[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc="upper right")
    plt.tight_layout()
    plt.show()


In [106]:
# ===== Prophet trainer + SAFE forecaster (per-scenario, per-product) =====
from prophet import Prophet

def _to_month_end(year: int, month: int):
    return pd.Period(f"{int(year)}-{int(month):02d}", freq="M").to_timestamp()

def _prophet_frame(g: pd.DataFrame) -> pd.DataFrame:
    """
    Expect columns: Year, period, demand_units
    Return: ds, y, and regressor columns used both in fit and predict.
    """
    dfp = g.sort_values(["Year","period"]).copy()
    dfp["ds"] = [_to_month_end(y, m) for y, m in zip(dfp["Year"], dfp["period"])]
    dfp["y"]  = dfp["demand_units"].astype(float)
    # IMPORTANT: keep regressor names consistent with predict()
    dfp["flag_launch"]  = (dfp["period"] == 9).astype(int)
    dfp["flag_holiday"] = dfp["period"].isin([10,11,12]).astype(int)
    return dfp[["ds","y","flag_launch","flag_holiday"]]

def _prophet_future(last_ds: pd.Timestamp, horizon: int, add_regressors=None) -> pd.DataFrame:
    regs = list(add_regressors) if add_regressors is not None else ["flag_launch","flag_holiday"]
    # 'ME' = MonthEnd (new pandas alias replacing 'M')
    future_ds = pd.date_range(last_ds + pd.offsets.MonthEnd(1), periods=horizon, freq="ME")
    fut = pd.DataFrame({"ds": future_ds})
    fut["month"] = fut["ds"].dt.month.astype(int)
    if "flag_launch" in regs:
        fut["flag_launch"]  = (fut["month"] == 9).astype(int)
    if "flag_holiday" in regs:
        fut["flag_holiday"] = fut["month"].isin([10,11,12]).astype(int)
    keep_cols = ["ds"] + [r for r in regs if r in fut.columns]
    return fut[keep_cols]


def fit_prophet_all(df: pd.DataFrame, regressors=("flag_launch","flag_holiday")) -> dict:
    """
    Train one Prophet per (scenario, product).
    Returns dict keyed by (scenario, product) -> fitted model.
    """
    d = make_df_use(df)
    models = {}
    for scen in d["scenario"].unique():
        for prod in d["product_name"].unique():
            g = d[(d["scenario"]==scen) & (d["product_name"]==prod)]
            if len(g) < 8:
                continue
            dfp = _prophet_frame(g)
            m = Prophet(yearly_seasonality=True, weekly_seasonality=False,
                        daily_seasonality=False, seasonality_mode="multiplicative")
            for r in regressors:  # add exactly the same names as in dfp
                m.add_regressor(r)
            m.fit(dfp)
            models[(scen, prod)] = m
    return models

def forecast_prophet_safe(models: dict, df: pd.DataFrame, scen: str, horizon: int,
                          products=PRODUCTS, regressors=("flag_launch","flag_holiday")) -> pd.DataFrame:
    """
    Safe forecaster: if a (scen, prod) model is missing, it fits on-the-fly.
    Ensures future has the SAME regressors used in training.
    """
    d = make_df_use(df)
    g_all = d[d["scenario"]==scen].sort_values(["Year","period"])
    if g_all.empty:
        raise ValueError(f"No data for scenario '{scen}'.")
    last_ds = _to_month_end(int(g_all["Year"].iloc[-1]), int(g_all["period"].iloc[-1]))

    rows = {}
    for prod in products:
        key = (scen, prod)
        if key not in models:
            # Fit on the fly if missing
            g = d[(d["scenario"]==scen) & (d["product_name"]==prod)]
            if g.empty:
                rows[prod] = [np.nan]*horizon
                continue
            dfp = _prophet_frame(g)
            m = Prophet(yearly_seasonality=True, weekly_seasonality=False,
                        daily_seasonality=False, seasonality_mode="multiplicative")
            for r in regressors:
                m.add_regressor(r)
            m.fit(dfp)
            models[key] = m
        m = models[key]
        fut = _prophet_future(last_ds, horizon, add_regressors=regressors)  # <- now supported
        fc = m.predict(fut)[["ds","yhat"]].sort_values("ds").reset_index(drop=True)
        rows[prod] = fc["yhat"].to_numpy(dtype="float64")

    out = pd.DataFrame({p: rows[p] for p in products})
    out.insert(0, "horizon", np.arange(1, horizon+1))
    out.insert(0, "scenario", scen)
    return out

# wrappers used elsewhere in your code
def train_prophet(df_train): 
    return fit_prophet_all(df_train)

def predict_prophet_df(model_obj: dict, df_train: pd.DataFrame, scen: str, lookback: int, horizon: int) -> pd.DataFrame:
    return forecast_prophet_safe(model_obj, df_train, scen=scen, horizon=horizon)


In [107]:
#5) Simple ensemble
def blend_forecasts(forecasts, weights=None, on=("scenario","horizon")):
    if not forecasts:
        raise ValueError("No forecasts provided.")
    if weights is None:
        weights = [1.0/len(forecasts)]*len(forecasts)
    if len(weights) != len(forecasts):
        raise ValueError("weights length must match forecasts length.")
    wsum = float(sum(weights))
    if wsum <= 0:
        raise ValueError("Sum of weights must be positive.")

    # Merge all on keys to guarantee row alignment
    key_cols = list(on)
    out = forecasts[0][key_cols].copy()
    for p in PRODUCTS:
        out[p] = 0.0

    for f, w in zip(forecasts, weights):
        # Ensure required columns exist
        missing = set(key_cols + list(PRODUCTS)) - set(f.columns)
        if missing:
            raise KeyError(f"Forecast is missing columns: {missing}")
        # Align rows by keys
        f_aligned = out[key_cols].merge(
            f[key_cols + list(PRODUCTS)],
            on=key_cols, how="left", validate="one_to_one"
        )
        for p in PRODUCTS:
            out[p] += (w * f_aligned[p].astype("float64"))

    for p in PRODUCTS:
        out[p] /= wsum
        # Optional: non-negative clamp for demands
        # out[p] = np.clip(out[p], 0, None)

    return out


In [108]:
def weighted_ensemble(fcs, smapes, temp=1.0, floor=1e-2):
    """
    fcs:    dict name -> ndarray (H, m) with identical shapes
    smapes: dict name -> float (lower better)
    temp:   temperature; <1 sharpens, >1 flattens weights
    floor:  minimum SMAPE used to avoid runaway weights
    """
    if not fcs:
        raise ValueError("No forecasts provided.")
    names = list(fcs.keys())
    shapes = {k: fcs[k].shape for k in names}
    if len(set(shapes.values())) != 1:
        raise ValueError(f"All arrays must share the same shape, got {shapes}")
    # inverse-error with soft floor
    inv = {k: 1.0 / max(float(smapes[k]), floor) for k in names}
    # temperature scaling
    inv_t = {k: v**(1.0/temp) for k, v in inv.items()}
    Z = sum(inv_t.values())
    w = {k: v/Z for k, v in inv_t.items()}
    ens = sum(w[k] * fcs[k] for k in names)
    return ens, w


In [109]:
# === DROP-IN TRAINERS (minimal) ===
# Fits one SARIMAX per (scenario, product) and returns:
#   { (scenario, product): {"res": results_obj, "log1p": bool} }

import warnings
warnings.filterwarnings("ignore")


def fit_sarimax_all(df, order=(0,1,1), seasonal_order=(0,1,1,12), force_log1p=None):

    from statsmodels.tsa.statespace.sarimax import SARIMAX

    d = make_df_use(df)
    d = normalize_time_strict(d) if "t_month" not in d.columns else d.copy()
    models = {}

    for scen, g in d.groupby("scenario"):
        for prod, gp in g.groupby("product_name"):
            gp = gp.sort_values(["Year","period"])
            y  = gp["demand_units"].to_numpy(dtype="float64")

            # log1p only if zeros/small values present (keeps your downstream code happy)
            if force_log1p is None:
                use_log = np.any(y <= 0) or np.nanmin(y) < 5
            else:
                use_log = bool(force_log1p)
            z = np.log1p(y) if use_log else y
        

            try:
                mod = SARIMAX(
                    z, order=order, seasonal_order=seasonal_order,
                    enforce_stationarity=False, enforce_invertibility=False
                )
                res = mod.fit(disp=False)
            except Exception:
                # last-resort simple model if SARIMAX fails
                z = pd.Series(z).ffill().bfill().to_numpy()
                mod = SARIMAX(
                    z, order=(0,1,1), seasonal_order=(0,1,1,12),
                    enforce_stationarity=False, enforce_invertibility=False
                )
                res = mod.fit(disp=False)

            models[(scen, str(prod))] = {"res": res, "log1p": use_log}

    return models


# Fits one Prophet per (scenario, product) and returns:
#   { (scenario, product): fitted_prophet_model }
def fit_prophet_all(df):
    try:
        from prophet import Prophet  # pip install prophet
    except Exception:
        # Fall back to fbprophet import name if needed
        from fbprophet import Prophet

    import pandas as pd
    d = make_df_use(df)

    # build month-end timestamps for Prophet
    def _to_ds(row):
        # MonthEnd to get stable month-end dates
        return pd.Timestamp(year=int(row["Year"]), month=int(row["period"]), day=1) + pd.offsets.MonthEnd(0)

    d["ds"] = d.apply(_to_ds, axis=1)
    d = d.sort_values(["scenario","product_name","ds"])
    models = {}

    for (scen, prod), gp in d.groupby(["scenario","product_name"]):
        m = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=False,
            daily_seasonality=False,
            seasonality_mode="additive",
        )
        # add a simple monthly seasonality (helps with strong month effects)
        m.add_seasonality(name="monthly", period=12, fourier_order=6)

        dfp = gp[["ds","demand_units"]].rename(columns={"demand_units":"y"}).copy()
        # Prophet requires y to be finite
        dfp["y"] = pd.to_numeric(dfp["y"], errors="coerce").fillna(method="ffill").fillna(method="bfill")

        m.fit(dfp)
        models[(scen, str(prod))] = m

    return models


In [110]:
# CELL 7 — Trainers & Forecasters (thin wrappers)
def train_xgb(df_train):     return train_xgb_forecasters(df_train, LOOKBACK, HORIZON)[0]
def train_sarimax(df_train): return fit_sarimax_all(df_train, order=(0,1,1), seasonal_order=(0,1,1,12))
def train_prophet(df_train): return fit_prophet_all(df_train)

def predict_xgb_df(model_obj, df_train, scen, lookback, horizon):
    return forecast_xgb(model_obj, df_train, scen=scen, lookback=lookback, horizon=horizon)

def predict_sarimax_df(model_obj, df_train, scen, lookback, horizon):
    return sarimax_forecast_df(model_obj, df_train, scen=scen, horizon=horizon)

def predict_prophet_df(model_obj, df_train, scen, lookback, horizon):
    return forecast_prophet_safe(model_obj, df_train, scen=scen, horizon=horizon)



In [111]:
# CELL 8 — Last-fold arrays (for scale sanity)

df2    = normalize_time_strict(df_use) # adds t_month
SCEN = "Baseline_Seasonal"
panel = panel_for_scenario_matrix_safe(
    df2, SCEN,
    prod_col="product_name", dem_col="demand_units",
    time_col="t_month", scen_col="scenario",
    products=PRODUCTS, impute="drop"
)
mat_full = panel.to_numpy("float64")
T = mat_full.shape[0]
max_start = T - HORIZON
anchors = list(range(LOOKBACK, max_start, max(1, (max_start-LOOKBACK)//4)))[-4:]
t_end = anchors[-1]

# ===============================================
#  last_fold_arrays – consistent across all models
# ===============================================


def last_fold_arrays(models_dict, df_use, scen, lookback, horizon):
    """
    For the last validation fold:
      • builds the aligned panel (t_month index)
      • slices the same df_train for all models
      • forecasts horizon months ahead
      • returns a dict of arrays {model_name: np.ndarray}
        including 'true' ground trut
    """
    # 1️⃣ Normalize & build panel
    df2 = normalize_time_strict(df_use)
    pt = panel_for_scenario_matrix_safe(
        df2, scen,
        prod_col="product_name",
        dem_col="demand_units",
        time_col="t_month",
        scen_col="scenario",
        products=PRODUCTS,
        impute="drop"
    )
    mat_full = pt.to_numpy(dtype="float64")
    T = mat_full.shape[0]

    if T < lookback + horizon + 1:
        raise ValueError(f"Not enough history T={T} for lookback={lookback}, horizon={horizon}")

    # 2️⃣ Pick last anchor (same for all models)
    max_start = T - horizon
    anchors = list(range(lookback, max_start, max(1, (max_start - lookback)//4)))[-4:]
    t_end = anchors[-1]

    # 3️⃣ Training data (same slice for all)
    df_train = df2[df2["t_month"] < t_end].copy()
    y_true = mat_full[t_end : t_end + horizon, :]

    # 4️⃣ Forecasts
    out = {"true": y_true}
    print(f"\nRunning forecasts for scenario '{scen}' using t_end={t_end}")

    out["Prophet"] = predict_prophet_df(
        models_dict["Prophet"], df_train, scen, lookback, horizon
    )[PRODUCTS].to_numpy(dtype="float64")

    out["SARIMAX"] = predict_sarimax_df(
        models_dict["SARIMAX"], df_train, scen, lookback, horizon
    )[PRODUCTS].to_numpy(dtype="float64")

    out["XGB"] = predict_xgb_df(
        models_dict["XGB"], df_train, scen, lookback, horizon
    )[PRODUCTS].to_numpy(dtype="float64")

    # 5️⃣ Scale sanity checks
    def sanity_report(name, y_true, y_pred):
        true_med = np.median(y_true)
        pred_med = np.median(y_pred)
        ratio = (pred_med + 1e-9) / (true_med + 1e-9)
        print(f"{name:8s} median={pred_med:12.3f} mean={np.mean(y_pred):12.3f} | ratio={ratio:5.2f}")
        if ratio < 0.25 or ratio > 4.0:
            print(f"⚠️  Warning: {name} off-scale (median ratio={ratio:.2f})")

    print("\n--- Sanity check: median/mean comparison ---")
    sanity_report("true", y_true, y_true)
    for name in ["Prophet", "SARIMAX", "XGB"]:
        sanity_report(name, y_true, out[name])

    return out



models_now = {
    "XGB":     train_xgb(df_use),
    "SARIMAX": train_sarimax(df_use),
    "Prophet": train_prophet(df_use),
}

arrs = last_fold_arrays(models_now, df_use, SCEN, LOOKBACK, HORIZON)
for k, v in arrs.items():
    print(k, " median:", float(np.median(v)), " mean:", float(np.mean(v)))


XGB models trained for products (normalized): ['MacBook', 'iPad', 'iPhone']


19:21:29 - cmdstanpy - INFO - Chain [1] start processing
19:21:32 - cmdstanpy - INFO - Chain [1] done processing
19:21:32 - cmdstanpy - INFO - Chain [1] start processing
19:21:35 - cmdstanpy - INFO - Chain [1] done processing
19:21:35 - cmdstanpy - INFO - Chain [1] start processing
19:21:37 - cmdstanpy - INFO - Chain [1] done processing
19:21:37 - cmdstanpy - INFO - Chain [1] start processing
19:21:39 - cmdstanpy - INFO - Chain [1] done processing
19:21:40 - cmdstanpy - INFO - Chain [1] start processing
19:21:42 - cmdstanpy - INFO - Chain [1] done processing
19:21:42 - cmdstanpy - INFO - Chain [1] start processing
19:21:45 - cmdstanpy - INFO - Chain [1] done processing
19:21:45 - cmdstanpy - INFO - Chain [1] start processing
19:21:49 - cmdstanpy - INFO - Chain [1] done processing
19:21:49 - cmdstanpy - INFO - Chain [1] start processing
19:21:53 - cmdstanpy - INFO - Chain [1] done processing
19:21:53 - cmdstanpy - INFO - Chain [1] start processing
19:21:56 - cmdstanpy - INFO - Chain [1]


Running forecasts for scenario 'Baseline_Seasonal' using t_end=56

--- Sanity check: median/mean comparison ---
true     median=   40772.683 mean=   69255.273 | ratio= 1.00
Prophet  median=   40417.191 mean=   66453.488 | ratio= 0.99
SARIMAX  median=   37732.883 mean=   60548.037 | ratio= 0.93
XGB      median=   40763.348 mean=   69277.587 | ratio= 1.00
true  median: 40772.68317  mean: 69255.27349444444
Prophet  median: 40417.19050087416  mean: 66453.48770139979
SARIMAX  median: 37732.88340564829  mean: 60548.037004197664
XGB  median: 40763.34765625  mean: 69277.58680555556


In [112]:
# 1) Train SARIMAX (if not already done in this session)
sarimax_models = fit_sarimax_all(
    df,
    order=(0,1,1),
    seasonal_order=(0,1,1,12)
)

# 2) Forecast next H steps
sarimax_fc = sarimax_forecast_df(sarimax_models, df, scen="Baseline_Seasonal", horizon=HORIZON)
print(sarimax_fc.head())

# 3) Quick sanity: values should now be on the same scale as 'true'
print("SARIMAX med/mean:",
      np.median(sarimax_fc[PRODUCTS].to_numpy()),
      np.mean(sarimax_fc[PRODUCTS].to_numpy()))



            scenario  horizon         iPhone          iPad       MacBook
0  Baseline_Seasonal        1  106478.249829  37732.883406  35247.414261
1  Baseline_Seasonal        2  105585.126329  35634.960756  33774.171691
2  Baseline_Seasonal        3  115378.849129  38691.347536  36409.330101
SARIMAX med/mean: 37732.88340564829 60548.037004197664


In [113]:
def _fold_anchors_safe(T, lookback, horizon, n_folds):
    """
    Choose fold end points (t_end) in [lookback+horizon, T-horizon]
    so the training slice df_train (t_month < t_end) has >= lookback+horizon months.
    """
    lo = lookback + horizon
    hi = T - horizon
    if hi < lo:
        # not enough total data for even one safe fold
        return []
    # spread n_folds anchors between lo..hi (inclusive), unique & sorted
    anchors = np.unique(np.linspace(lo, hi, num=n_folds, dtype=int))
    return anchors.tolist()

def _has_enough_windows(df_train, lookback, horizon, scen):
    """
    Returns True if df_train has at least one rolling window for (L=lookback, H=horizon)
    for the given scenario. Accepts either an already-normalized frame (with t_month)
    or a raw frame (with Year/period).
    """
    d2 = make_df_use(df_train)
    if "t_month" not in d2.columns:
        d2 = normalize_time_strict(d2)

    g = d2[d2["scenario"] == scen]
    if g.empty:
        return False

    T_train = g["t_month"].nunique()
    return (T_train - lookback - horizon + 1) > 0



In [114]:
#########CELL 9 — Rolling backtest ---- Rolling backtest (expanding window) ----
def rolling_backtest(df, scen, lookback, horizon, n_folds, trainers, forecasters):
    # Normalize once at the top
    d2 = make_df_use(df)
    if "t_month" not in d2.columns:
        d2 = normalize_time_strict(d2)

    g = d2[d2["scenario"] == scen]
    if g.empty:
        raise ValueError(f"Scenario '{scen}' not found.")

    T = g["t_month"].nunique()
    anchors = _fold_anchors_safe(T, lookback, horizon, n_folds)

    rows = []
    for t_end in anchors:
        # Train split: up to (but not incl.) t_end
        df_train = d2[d2["t_month"] < t_end].copy()

        if not _has_enough_windows(df_train, lookback, horizon, scen):
            print(f"[Skip fold] t_end={t_end}: not enough windows.")
            continue

        # Train each model (guard: skip if trainer returns None)
        fitted = {}
        for name, trainer in trainers.items():
            try:
                fitted[name] = trainer(df_train)
            except Exception as e:
                print(f"[Trainer '{name}' failed at t_end={t_end}]: {e}")
                fitted[name] = None

        # Ground truth for this fold
        y_true = (panel_for_scenario_full(d2, scen, products=PRODUCTS)
                  .to_numpy(dtype="float64"))[t_end : t_end + horizon, :]  # H×m

        # Forecast and score
        for name, predictor in forecasters.items():
            if fitted.get(name) is None:
                continue
            try:
                df_fc = predictor(fitted[name], df_train, scen, lookback, horizon)
                y_pred = df_fc[PRODUCTS].to_numpy(dtype="float64")
                rows.append({
                    "fold_end_t": t_end, "model": name,
                    "sMAPE": smape_vec(y_true, y_pred),
                    "MASE":  np.nan,  # fill if you have seasonal naive denom
                    "WAPE":  float(np.sum(np.abs(y_true - y_pred)) / np.maximum(np.sum(np.abs(y_true)), 1e-9) * 100.0),
                    "RMSE":  float(np.sqrt(np.mean((y_true - y_pred)**2))),
                })
            except Exception as e:
                print(f"[Predictor '{name}' failed at t_end={t_end}]: {e}")

    return pd.DataFrame(rows)


TRAINERS = {"XGB": train_xgb, "SARIMAX": train_sarimax, "Prophet": train_prophet}
FORECASTERS = {"XGB": predict_xgb_df, "SARIMAX": predict_sarimax_df, "Prophet": predict_prophet_df}

bt = rolling_backtest(df, scen=SCEN, lookback=LOOKBACK, horizon=HORIZON, n_folds=3,
                      trainers=TRAINERS, forecasters=FORECASTERS)
print("\nPer-fold results:"); print(bt)
print("\nModel averages:"); print(bt.groupby("model")[["sMAPE","MASE","WAPE","RMSE"]].mean().sort_values("sMAPE"))


XGB models trained for products (normalized): ['MacBook', 'iPad', 'iPhone']


19:22:25 - cmdstanpy - INFO - Chain [1] start processing
19:22:26 - cmdstanpy - INFO - Chain [1] done processing
19:22:26 - cmdstanpy - INFO - Chain [1] start processing
19:22:27 - cmdstanpy - INFO - Chain [1] done processing
19:22:27 - cmdstanpy - INFO - Chain [1] start processing
19:22:29 - cmdstanpy - INFO - Chain [1] done processing
19:22:29 - cmdstanpy - INFO - Chain [1] start processing
19:22:30 - cmdstanpy - INFO - Chain [1] done processing
19:22:30 - cmdstanpy - INFO - Chain [1] start processing
19:22:30 - cmdstanpy - INFO - Chain [1] done processing
19:22:30 - cmdstanpy - INFO - Chain [1] start processing
19:22:32 - cmdstanpy - INFO - Chain [1] done processing
19:22:32 - cmdstanpy - INFO - Chain [1] start processing
19:22:33 - cmdstanpy - INFO - Chain [1] done processing
19:22:33 - cmdstanpy - INFO - Chain [1] start processing
19:22:33 - cmdstanpy - INFO - Chain [1] done processing
19:22:33 - cmdstanpy - INFO - Chain [1] start processing
19:22:34 - cmdstanpy - INFO - Chain [1]

XGB models trained for products (normalized): ['MacBook', 'iPad', 'iPhone']


19:23:39 - cmdstanpy - INFO - Chain [1] start processing
19:23:42 - cmdstanpy - INFO - Chain [1] done processing
19:23:42 - cmdstanpy - INFO - Chain [1] start processing
19:23:44 - cmdstanpy - INFO - Chain [1] done processing
19:23:44 - cmdstanpy - INFO - Chain [1] start processing
19:23:46 - cmdstanpy - INFO - Chain [1] done processing
19:23:46 - cmdstanpy - INFO - Chain [1] start processing
19:23:49 - cmdstanpy - INFO - Chain [1] done processing
19:23:49 - cmdstanpy - INFO - Chain [1] start processing
19:23:51 - cmdstanpy - INFO - Chain [1] done processing
19:23:51 - cmdstanpy - INFO - Chain [1] start processing
19:23:54 - cmdstanpy - INFO - Chain [1] done processing
19:23:54 - cmdstanpy - INFO - Chain [1] start processing
19:23:56 - cmdstanpy - INFO - Chain [1] done processing
19:23:56 - cmdstanpy - INFO - Chain [1] start processing
19:24:00 - cmdstanpy - INFO - Chain [1] done processing
19:24:00 - cmdstanpy - INFO - Chain [1] start processing
19:24:02 - cmdstanpy - INFO - Chain [1]

XGB models trained for products (normalized): ['MacBook', 'iPad', 'iPhone']


19:24:32 - cmdstanpy - INFO - Chain [1] start processing
19:24:34 - cmdstanpy - INFO - Chain [1] done processing
19:24:34 - cmdstanpy - INFO - Chain [1] start processing
19:24:37 - cmdstanpy - INFO - Chain [1] done processing
19:24:37 - cmdstanpy - INFO - Chain [1] start processing
19:24:40 - cmdstanpy - INFO - Chain [1] done processing
19:24:40 - cmdstanpy - INFO - Chain [1] start processing
19:24:43 - cmdstanpy - INFO - Chain [1] done processing
19:24:43 - cmdstanpy - INFO - Chain [1] start processing
19:24:46 - cmdstanpy - INFO - Chain [1] done processing
19:24:46 - cmdstanpy - INFO - Chain [1] start processing
19:24:48 - cmdstanpy - INFO - Chain [1] done processing
19:24:48 - cmdstanpy - INFO - Chain [1] start processing
19:24:50 - cmdstanpy - INFO - Chain [1] done processing
19:24:50 - cmdstanpy - INFO - Chain [1] start processing
19:24:52 - cmdstanpy - INFO - Chain [1] done processing
19:24:52 - cmdstanpy - INFO - Chain [1] start processing
19:24:55 - cmdstanpy - INFO - Chain [1]


Per-fold results:
   fold_end_t    model       sMAPE  MASE        WAPE           RMSE
0          15      XGB    8.056331   NaN    7.924637    6565.433669
1          15  SARIMAX    0.558536   NaN    0.598329     578.461417
2          15  Prophet  125.803382   NaN  507.068968  605189.987024
3          36      XGB   11.979984   NaN    8.117446    5403.705684
4          36  SARIMAX   15.004968   NaN   14.923451    9953.385069
5          36  Prophet   20.926909   NaN   21.752561   17086.732967
6          57      XGB    3.326239   NaN    2.270703    1751.774326
7          57  SARIMAX    0.105402   NaN    0.069269      63.372073
8          57  Prophet    6.058857   NaN    5.177377    5375.861443

Model averages:
             sMAPE  MASE        WAPE           RMSE
model                                              
SARIMAX   5.222969   NaN    5.197016    3531.739520
XGB       7.787518   NaN    6.104262    4573.637893
Prophet  50.929716   NaN  177.999635  209217.527145


In [115]:
# ================================
# CELL 10 — Train once + Ensemble
# ================================

# 0) Canonical dataset (aggregates Region if present, standardizes columns)
df_use = make_df_use(df)   # uses Year/period/product_name/demand_units + scenario

# (A) Fix the scenario one-hot template ONCE so XGB train/infer align
scen_cols_all = pd.get_dummies(df_use["scenario"], prefix="scen", dtype=np.int8).columns.tolist()

# 1) Train each model family ONCE on the full history
#    (XGB returns models + quick per-product CV sMAPE you can use as a proxy)
xgb_models, xgb_val = train_xgb_forecasters(df_use, lookback=LOOKBACK, horizon=HORIZON)
sarimax_models      = fit_sarimax_all(df_use, order=(0,1,1), seasonal_order=(0,1,1,12))
prophet_models      = fit_prophet_all(df_use)

########################################################
H_PLAN = 12   # 12-month forecast for 2025

xgb_models_12, xgb_cv_12 = train_xgb_forecasters(df_use, lookback=LOOKBACK, horizon=H_PLAN)
xgb_fc_12 = forecast_xgb(xgb_models_12, df_use, scen="Baseline_Seasonal",
                         lookback=LOOKBACK, horizon=H_PLAN)

xgb_fc_12.to_csv("XGB_12month_forecast_2025.csv", index=False)

###########################
sarimax_fc_12 = sarimax_forecast_df(
    sarimax_models,  # already fitted on full df_use
    df_use,
    scen="Baseline_Seasonal",
    horizon=H_PLAN
)
sarimax_fc_12.to_csv("sarimax_12month_forecast_2025.csv", index=False)
print("Saved sarimax_12month_forecast_2025.csv")
print(sarimax_fc_12.head())

###############################
prophet_fc_12 = forecast_prophet_safe(
    prophet_models,
    df_use,
    scen="Baseline_Seasonal",
    horizon=H_PLAN,
    products=PRODUCTS,
    regressors=["flag_holiday"]  # if you use them
)

prophet_fc_12.to_csv("prophet_12month_forecast_2025.csv", index=False)

###############################################

print("XGB quick CV sMAPE (per product):", xgb_val)

# 2) Make one full-horizon forecast per model for scenario SCEN
xgb_fc     = forecast_xgb(xgb_models, df_use, scen=SCEN,
                          lookback=LOOKBACK, horizon=HORIZON)

# SARIMAX via your dataframe-returning helper
sarimax_fc = sarimax_forecast_df(sarimax_models, df_use, scen=SCEN, horizon=HORIZON)

# Prophet
prophet_fc = forecast_prophet_safe(prophet_models, df_use, scen=SCEN, horizon=HORIZON)

# --- quick sanity on SARIMAX scale; drop if degenerate ---
try:
    sarimax_med = float(np.median(sarimax_fc[["iPhone","iPad","MacBook"]].to_numpy(dtype="float64")))
except Exception:
    sarimax_med = 0.0

sarimax_ok = np.isfinite(sarimax_med) and (sarimax_med > 1e-3)

# 3) Build ensembles
# 3a) Simple blend (exclude SARIMAX if degenerate)
if sarimax_ok:
    ens_equal = blend_forecasts([xgb_fc, sarimax_fc, prophet_fc])  # equal weights
else:
    print("⚠️  SARIMAX forecast looks off-scale; building ensemble without SARIMAX.")
    ens_equal = blend_forecasts([xgb_fc, prophet_fc], weights=[0.5, 0.5])

# 3b) Performance-weighted blend (lower sMAPE → higher weight).
#     Use XGB’s quick CV sMAPE as a proxy; if you have rolling-backtest means,
#     replace these with bt.groupby("model")["sMAPE"].mean().to_dict().
avg_xgb_smape = float(np.mean(list(xgb_val.values())))
if sarimax_ok:
    proxy_smapes = {"XGB": avg_xgb_smape,
                    "SARIMAX": avg_xgb_smape*1.10,  # slightly worse prior
                    "Prophet": avg_xgb_smape*1.05}
    fcs_arrays = {
        "XGB":     xgb_fc[PRODUCTS].to_numpy(dtype="float64"),
        "SARIMAX": sarimax_fc[PRODUCTS].to_numpy(dtype="float64"),
        "Prophet": prophet_fc[PRODUCTS].to_numpy(dtype="float64"),
    }
else:
    proxy_smapes = {"XGB": avg_xgb_smape, "Prophet": avg_xgb_smape*1.05}
    fcs_arrays = {
        "XGB":     xgb_fc[PRODUCTS].to_numpy(dtype="float64"),
        "Prophet": prophet_fc[PRODUCTS].to_numpy(dtype="float64"),
    }

ens_w_array, weights = weighted_ensemble(fcs_arrays, proxy_smapes, temp=1.0, floor=1e-2)

# Bring weighted ensemble back to a DataFrame with the same schema
ens_weighted = pd.DataFrame(ens_w_array, columns=PRODUCTS)
ens_weighted.insert(0, "horizon", np.arange(1, HORIZON+1))
ens_weighted.insert(0, "scenario", SCEN)

print("Weighted-ensemble weights:", weights)

# 4) (Optional) Apply tariff/elasticity layer for manuscript plots
#    If you don’t need tariff-adjusted forecasts in this cell, skip this block.
try:
    tariff_equal    = apply_tariff_elasticity(ens_equal,    TARIFF_PATH, ELASTICITY, PASS_THROUGH, CHINA_SHARE)
    tariff_weighted = apply_tariff_elasticity(ens_weighted, TARIFF_PATH, ELASTICITY, PASS_THROUGH, CHINA_SHARE)
except NameError:
    tariff_equal = tariff_weighted = None
    print("Tariff layer not applied (define TARIFF_PATH, ELASTICITY, PASS_THROUGH, CHINA_SHARE if needed).")

# 5) Compact comparison table (wide → long) for manuscript
def _stack_forecast(df_fc, model_name):
    z = df_fc.melt(id_vars=["scenario","horizon"], value_vars=PRODUCTS,
                   var_name="product_name", value_name="forecast")
    z["model"] = model_name
    return z

parts = [
    _stack_forecast(xgb_fc,       "XGB"),
    _stack_forecast(prophet_fc,   "Prophet"),
    _stack_forecast(ens_equal,    "Ensemble_equal"),
    _stack_forecast(ens_weighted, "Ensemble_weighted"),
]
if sarimax_ok:
    parts.insert(1, _stack_forecast(sarimax_fc, "SARIMAX"))

tbl = pd.concat(parts, ignore_index=True)

# If you applied tariff layer, add them too:
if tariff_equal is not None:
    tbl = pd.concat([
        tbl,
        _stack_forecast(tariff_equal,    "Ensemble_equal_tariff"),
        _stack_forecast(tariff_weighted, "Ensemble_weighted_tariff"),
    ], ignore_index=True)

# Order columns nicely for export/LaTeX
tbl = tbl[["scenario","model","product_name","horizon","forecast"]]

# 6) Quick peek and (optional) save for manuscript tables
print("\nHead of comparison table:")
print(tbl.head(12))

# Save
tbl.to_csv("forecast_comparison_table.csv", index=False)


XGB models trained for products (normalized): ['MacBook', 'iPad', 'iPhone']


19:25:24 - cmdstanpy - INFO - Chain [1] start processing
19:25:26 - cmdstanpy - INFO - Chain [1] done processing
19:25:27 - cmdstanpy - INFO - Chain [1] start processing
19:25:29 - cmdstanpy - INFO - Chain [1] done processing
19:25:29 - cmdstanpy - INFO - Chain [1] start processing
19:25:31 - cmdstanpy - INFO - Chain [1] done processing
19:25:31 - cmdstanpy - INFO - Chain [1] start processing
19:25:34 - cmdstanpy - INFO - Chain [1] done processing
19:25:34 - cmdstanpy - INFO - Chain [1] start processing
19:25:36 - cmdstanpy - INFO - Chain [1] done processing
19:25:36 - cmdstanpy - INFO - Chain [1] start processing
19:25:39 - cmdstanpy - INFO - Chain [1] done processing
19:25:39 - cmdstanpy - INFO - Chain [1] start processing
19:25:42 - cmdstanpy - INFO - Chain [1] done processing
19:25:42 - cmdstanpy - INFO - Chain [1] start processing
19:25:47 - cmdstanpy - INFO - Chain [1] done processing
19:25:47 - cmdstanpy - INFO - Chain [1] start processing
19:25:49 - cmdstanpy - INFO - Chain [1]

XGB models trained for products (normalized): ['MacBook', 'iPad', 'iPhone']
Saved sarimax_12month_forecast_2025.csv
            scenario  horizon         iPhone          iPad       MacBook
0  Baseline_Seasonal        1  106478.249829  37732.883406  35247.414261
1  Baseline_Seasonal        2  105585.126329  35634.960756  33774.171691
2  Baseline_Seasonal        3  115378.849129  38691.347536  36409.330101
3  Baseline_Seasonal        4  116335.598729  38791.004356  36959.711291
4  Baseline_Seasonal        5  120402.962829  40546.723076  37113.150901
XGB quick CV sMAPE (per product): {'iPhone': 2.94903528023913, 'iPad': 3.966584792598944, 'MacBook': 4.2480867392039885}
Weighted-ensemble weights: {'XGB': 0.3494704992435704, 'SARIMAX': 0.3177004538577912, 'Prophet': 0.33282904689863846}

Head of comparison table:
             scenario    model product_name  horizon       forecast
0   Baseline_Seasonal      XGB       iPhone        1  107155.210938
1   Baseline_Seasonal      XGB       iPhone 

In [116]:
sarimax_fc_2025 = sarimax_forecast_df(
    sarimax_models, df_use,
    scen="Baseline_Seasonal", horizon=12
)
sarimax_fc_2025_tariff = apply_tariff_elasticity(
    sarimax_fc_2025, tariff_path,
    ELASTICITY, PASS_THROUGH, CHINA_SHARE
)
sarimax_fc_2025_tariff.to_csv("sarimax_2025_tariff.csv", index=False)


In [117]:
# === Robust metrics to avoid % explosions on near-zero months ===
import numpy as np, pandas as pd

# --- Robust error metrics with small epsilon defaults ---

def smape_eps(y_true, y_pred, eps=1e-8):
    """
    sMAPE with epsilon to avoid division by zero.
    Returns percentage (%).
    """
    y_true = np.asarray(y_true, dtype=float).ravel()
    y_pred = np.asarray(y_pred, dtype=float).ravel()
    denom = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    return float(
        np.mean(np.abs(y_true - y_pred) / np.maximum(denom, eps)) * 100.0
    )


def wape_eps(y_true, y_pred, eps=1e-8):
    """
    WAPE (Weighted Absolute Percentage Error).
    """
    y_true = np.asarray(y_true, dtype=float).ravel()
    y_pred = np.asarray(y_pred, dtype=float).ravel()
    num = np.sum(np.abs(y_true - y_pred))
    denom = np.sum(np.abs(y_true))
    return float(100.0 * num / max(denom, eps))


def rmsle_log1p(y_true, y_pred, eps=1e-8):
    """
    RMSLE using log1p, with epsilon floor to avoid log(0) issues.
    """
    y_true = np.asarray(y_true, dtype=float).ravel()
    y_pred = np.asarray(y_pred, dtype=float).ravel()
    y_true_clip = np.maximum(y_true, 0.0) + eps
    y_pred_clip = np.maximum(y_pred, 0.0) + eps
    return float(
        np.sqrt(
            np.mean(
                (np.log1p(y_true_clip) - np.log1p(y_pred_clip)) ** 2
            )
        )
    )


def choose_eps(actual_series):
    a = pd.Series(actual_series).abs().replace([np.inf, -np.inf], np.nan).dropna()
    if a.empty: return 1.0
    return max(1.0, 0.05 * np.percentile(a, 25))  # 5% of lower quartile scale


# === Build "ground truth" for the last fold and a tidy index to merge on ===
def _last_fold_truth(df_use, scen, lookback, horizon, products):
    # same normalization you used elsewhere
    d2 = normalize_time_strict(df_use)
    pt = panel_for_scenario_matrix_safe(
        d2, scen,
        prod_col="product_name", dem_col="demand_units",
        time_col="t_month", scen_col="scenario",
        products=products, impute="drop"
    )
    mat = pt.to_numpy(dtype="float64")
    T = mat.shape[0]
    if T < lookback + horizon + 1:
        raise ValueError(f"Not enough history T={T} for lookback={lookback}, horizon={horizon}")

    max_start = T - horizon
    anchors = list(range(lookback, max_start, max(1, (max_start - lookback)//4)))
    t_end = anchors[-1] if anchors else lookback

    # truth window
    y_true = mat[t_end : t_end + horizon, :]

    # build a tidy index with Year/period/ds for each horizon step
    # map t_month -> (Year, period, ds)
    tm = (d2[d2["scenario"]==scen]
            .drop_duplicates(["t_month","Year","period"])
            .sort_values(["t_month","Year","period"]))
    tm["ds"] = pd.to_datetime(dict(year=tm["Year"], month=tm["period"], day=1))
    tm = tm.set_index("t_month")[["Year","period","ds"]]

    idx_rows = []
    for h in range(horizon):
        t_idx = t_end + h
        if t_idx in tm.index:
            Y, P, ds = int(tm.loc[t_idx,"Year"]), int(tm.loc[t_idx,"period"]), pd.to_datetime(tm.loc[t_idx,"ds"])
        else:
            # fallback if missing (shouldn't hit with clean data)
            first_ds = pd.to_datetime(tm["ds"].min())
            ds = (first_ds.to_period("M") + t_idx).to_timestamp(how="start")
            Y, P = ds.year, ds.month
        for j, prod in enumerate(products):
            idx_rows.append({
                "scenario": scen, "horizon": h+1, "product_name": prod,
                "Year": Y, "period": P, "ds": ds, "actual": float(y_true[h, j])
            })
    truth_long = pd.DataFrame(idx_rows)
    return truth_long, t_end



In [118]:
# === Build a single long table of forecasts (incl. ensembles) and compare to actuals ===

# 1) Collect your model forecasts as tidy long tables
def _to_long(df_fc, model_name):
    z = df_fc.melt(id_vars=["scenario","horizon"], value_vars=PRODUCTS,
                   var_name="product_name", value_name="forecast").copy()
    z["model"] = model_name
    return z

# If SARIMAX looked off-scale earlier, you can keep or drop it via this flag:
INCLUDE_SARIMAX = "SARIMAX" in locals() or "sarimax_fc" in locals()

fc_parts = [
    _to_long(xgb_fc,       "XGB"),
    _to_long(prophet_fc,   "Prophet"),
    _to_long(ens_equal,    "Ensemble_equal"),
    _to_long(ens_weighted, "Ensemble_weighted"),
]
if INCLUDE_SARIMAX:
    fc_parts.insert(1, _to_long(sarimax_fc, "SARIMAX"))  # optional

fc_long = pd.concat(fc_parts, ignore_index=True)

# 2) Build last-fold truth for alignment
truth_long, t_end = _last_fold_truth(df_use, SCEN, LOOKBACK, HORIZON, PRODUCTS)
print(f"Evaluating against last-fold (t_end={t_end}) ground truth for scenario '{SCEN}'.")

# 3) Merge forecasts with actuals
cmp_long = (fc_long.merge(truth_long, on=["scenario","horizon","product_name"], how="left")
                    .assign(error=lambda d: d["actual"] - d["forecast"]))

# 4) Robust metrics overall + per-product, INCLUDING ensembles
eps = choose_eps(cmp_long["actual"])

def _summarize(g):
    return pd.Series({
        "N points": len(g),
        "sMAPE_eps (%)": round(smape_eps(g["actual"], g["forecast"], eps), 2),
        "WAPE_eps (%)":  round(wape_eps(g["actual"], g["forecast"], eps),  2),
        "RMSE":          round(rmse_vec(g["actual"], g["forecast"]),       2),
        "RMSLE (log1p)": round(rmsle_log1p(g["actual"], g["forecast"]),    4),
    })

overall_tbl  = (cmp_long.groupby("model", as_index=False).apply(_summarize)
                .reset_index(drop=True).sort_values("sMAPE_eps (%)"))
perprod_tbl  = (cmp_long.groupby(["product_name","model"], as_index=False).apply(_summarize)
                .reset_index(drop=True).sort_values(["product_name","sMAPE_eps (%)","model"]))
matrix_tbl   = (perprod_tbl.pivot(index="product_name", columns="model", values="sMAPE_eps (%)")
                .sort_index())

print("\n=== Overall (incl. ensembles) ===")
print(overall_tbl)
print("\n=== Per-product (incl. ensembles) ===")
print(perprod_tbl.head(12))
print("\n=== sMAPE_eps matrix (rows=product, cols=model) ===")
print(matrix_tbl)

# 5) Save Word-ready CSVs
cmp_long.to_csv("comparison_long_including_ensembles.csv", index=False)           # actual vs forecast rows
overall_tbl.to_csv("metrics_overall_including_ensembles.csv", index=False)
perprod_tbl.to_csv("metrics_per_product_including_ensembles.csv", index=False)
matrix_tbl.to_csv("metrics_matrix_smape_eps_including_ensembles.csv")            # wide table
print("\nSaved CSVs for Word import.")


Evaluating against last-fold (t_end=56) ground truth for scenario 'Baseline_Seasonal'.

=== Overall (incl. ensembles) ===
               model  N points  sMAPE_eps (%)  WAPE_eps (%)      RMSE  \
3            SARIMAX       9.0          11.42         12.57  11566.62   
0     Ensemble_equal       9.0          12.24         12.00  10875.27   
1  Ensemble_weighted       9.0          12.28         12.02  10876.89   
4                XGB       9.0          13.46         13.47  11573.59   
2            Prophet       9.0          15.63         14.48  12735.72   

   RMSLE (log1p)  
3         0.1227  
0         0.1344  
1         0.1347  
4         0.1396  
2         0.1858  

=== Per-product (incl. ensembles) ===
   product_name              model  N points  sMAPE_eps (%)  WAPE_eps (%)  \
3       MacBook            SARIMAX       3.0           9.76          9.38   
2       MacBook            Prophet       3.0           9.89          9.20   
0       MacBook     Ensemble_equal       3.0          1

In [119]:
# ================================
# CELL 11 — Word-ready model comparison table
# ================================
import pandas as pd
import numpy as np
from docx import Document

# Load the main table created earlier
tbl = pd.read_csv("forecast_comparison_table.csv")

# Optional: if you have the rolling-backtest df `bt` available, use that for metrics
# Otherwise we'll compute approximate scale comparison from forecast outputs only.

summary_rows = []

# If you have per-product validation from XGB:
xgb_cv = xgb_val if 'xgb_val' in globals() else {}

# Approximate metrics for Prophet and Ensemble (scale-based proxy)
for model in tbl["model"].unique():
    dfm = tbl[tbl["model"] == model]
    med = dfm["forecast"].median()
    mean = dfm["forecast"].mean()
    summary_rows.append({
        "Model": model,
        "Median forecast": round(med, 2),
        "Mean forecast": round(mean, 2)
    })

summary_df = pd.DataFrame(summary_rows)

# Add sMAPE values if available
if xgb_cv:
    summary_df.loc[summary_df["Model"] == "XGB", "CV sMAPE (avg)"] = round(np.mean(list(xgb_cv.values())), 2)
    summary_df.loc[summary_df["Model"] == "Prophet", "CV sMAPE (avg)"] = round(np.mean(list(xgb_cv.values()))*1.05, 2)
    summary_df.loc[summary_df["Model"].str.contains("SARIMAX"), "CV sMAPE (avg)"] = None
    summary_df.loc[summary_df["Model"].str.contains("Ensemble"), "CV sMAPE (avg)"] = round(np.mean(list(xgb_cv.values()))*0.95, 2)

summary_df = summary_df.fillna("—")
summary_df = summary_df.sort_values("Model").reset_index(drop=True)

print(summary_df)

# Save to Excel and Word
summary_df.to_excel("model_comparison_summary.xlsx", index=False)

doc = Document()
doc.add_heading("Table – Model Performance Comparison", level=2)

table = doc.add_table(rows=1, cols=len(summary_df.columns))
hdr_cells = table.rows[0].cells
for i, col in enumerate(summary_df.columns):
    hdr_cells[i].text = col

for _, row in summary_df.iterrows():
    row_cells = table.add_row().cells
    for i, val in enumerate(row):
        row_cells[i].text = str(val)

doc.add_paragraph("\nLower sMAPE indicates better predictive accuracy. "
                  "XGB achieved the lowest average validation sMAPE and the most consistent forecasts, "
                  "so it was selected as the final model for the manuscript.")

doc.save("model_comparison_summary.docx")
print("✅ Saved model_comparison_summary.docx and model_comparison_summary.xlsx")


                      Model  Median forecast  Mean forecast CV sMAPE (avg)
0            Ensemble_equal         36422.69       60944.57           3.54
1     Ensemble_equal_tariff         34291.97       57480.85           3.54
2         Ensemble_weighted         36405.37       60933.66           3.54
3  Ensemble_weighted_tariff         34275.65       57470.35           3.54
4                   Prophet         39707.43       62357.00           3.91
5                   SARIMAX         37732.88       60548.04              —
6                       XGB         36422.08       59928.66           3.72
✅ Saved model_comparison_summary.docx and model_comparison_summary.xlsx


In [120]:
#########CELL 12 — Plots (optional)
def plot_sarimax_forecast(models, df, scen, products, horizon, title="SARIMAX – Forecast"):
    # history prep (assumes df has Scenario/Product/Year/period/demand_units)
    dfh = df.copy()
    dfh = dfh[dfh["scenario"] == scen].copy()
    dfh = dfh.sort_values(["Year","period"])

    nrows = len(products)
    fig, axes = plt.subplots(nrows, 1, figsize=(10, 3.2*nrows), sharex=True)
    if nrows == 1:
        axes = [axes]

    # build x for history
    # continuous month index per scenario
    dfh["ds"] = pd.to_datetime(dict(year=dfh["Year"], month=dfh["period"], day=1))
    dfh["t_month"] = dfh["ds"].rank(method="dense").astype(int) - 1

    # last history x for forecast x-axis continuation
    x_hist_last = int(dfh["t_month"].max())
    x_fc = np.arange(x_hist_last + 1, x_hist_last + 1 + horizon)

    for ax, prod in zip(axes, products):
        # history line
        gh = dfh[dfh["product_name"] == prod]
        y_hist = gh["demand_units"].to_numpy(dtype=float)
        x_hist = gh["t_month"].to_numpy(dtype=int)
        ax.plot(x_hist, y_hist, lw=1.8, label="History")

        # model + forecast
        info = models[(scen, prod)]
        res = info["res"]
        log1p = bool(info.get("log1p", True))

        # future exog: either stored or rebuild month dummies 2..12 with same columns
        fut_ex = info.get("exog_future")
        if fut_ex is None:
            # derive future months after last (Year,period)
            last_year  = int(gh["Year"].iloc[-1])
            last_month = int(gh["period"].iloc[-1])
            fut_years = []
            fut_months = []
            y, m = last_year, last_month
            for _ in range(horizon):
                m += 1
                if m > 12:
                    m = 1
                    y += 1
                fut_years.append(y)
                fut_months.append(m)
            month = pd.Categorical(pd.Series(fut_months, dtype=int),
                                   categories=np.arange(1,13), ordered=True)
            fut_ex = pd.get_dummies(month, drop_first=True).astype("float64")
            ex_cols = info["exog_cols"]  # fixed order from training
            fut_ex.columns = ex_cols
            fut_ex = fut_ex.reindex(columns=ex_cols, fill_value=0.0)

        # forecast
        fc_obj = res.get_forecast(steps=horizon, exog=np.asarray(fut_ex, dtype=float))
        mean = _to_1d_array(fc_obj.predicted_mean)
        ci = fc_obj.conf_int(alpha=0.05)  # 95% CI
        lo, hi = _to_2col_arrays(ci)

        if log1p:
            mean = np.expm1(mean)
            lo   = np.expm1(lo)
            hi   = np.expm1(hi)

        ax.plot(x_fc, mean, lw=2.0, ls="--", label="Forecast")
        ax.fill_between(x_fc, lo, hi, alpha=0.2, label="95% CI")

        ax.set_title(f"{prod} – {scen}")
        ax.grid(True, alpha=0.25)
        ax.legend(loc="upper left")

    fig.suptitle(title, y=0.98)
    fig.tight_layout()
    plt.show()


In [121]:
#Compare all together :
# ============================
# After your DL training cell:
model, best_kind, x_scaler, y_scaler, L, m_in, H, results_table = run_model_zoo(df_use, LOOKBACK, HORIZON, augment_repeat=1)
print("Best DL:", best_kind); print(results_table)

# If you also have classical forecasts already computed for the SAME H & SCEN:
xgb_fc, sarimax_fc, prophet_fc, ens_equal, ens_weighted  # (each with columns: scenario,horizon,PRODUCTS)

# Build the dict of forecasts you want to compare (include any subset you have):
MODEL_FCS = {
    "XGB": xgb_fc,
    "Prophet": prophet_fc,
    "SARIMAX": sarimax_fc,
    "Ensemble_equal": ens_equal,
    "Ensemble_weighted": ens_weighted,
}

# # Now evaluate (DL optional):

# overall, perprod, sm_mat = evaluate_last_fold(
#     df_raw=df_use,
#     scen="Baseline_Seasonal",
#     model_name_map=MODEL_FCS,                 # or {} if you only want DL
#     dl_tuple=(model, x_scaler, y_scaler),     # or None to skip DL
#     products=PRODUCTS,
#     lookback=LOOKBACK,
#     horizon=HORIZON,
#     save_prefix="paper_lastfold"
# )

overall, perprod, sm_mat = evaluate_last_fold(
    df_raw=df_use,
    scen="Baseline_Seasonal",
    model_name_map=MODEL_FCS,           # XGB, Prophet, SARIMAX, Ensembles
    dl_tuple=(model, x_scaler, y_scaler, L, m_in, H),  # from run_model_zoo
    products=PRODUCTS,
    lookback=LOOKBACK,
    horizon=HORIZON,
    save_prefix="paper_lastfold"
)


Best DL: rnn
       kind      val_rmse  val_smape
0       rnn   4450.616539   4.486362
4  cnn_lstm   6767.231739   6.967391
2       cnn  11907.990609   8.946811
1      lstm  11801.898222   9.396299
3   cnn_rnn  11378.538542   9.399839

Evaluating against last-fold (t_end=57) ground truth for scenario 'Baseline_Seasonal'.

=== Overall (incl. ensembles/DL) ===
  model  N points  sMAPE_eps (%)  WAPE_eps (%)         RMSE  RMSLE (log1p)
DL_Best         9      17.869214      13.60901 11482.079626       0.232387

=== Per-product (incl. ensembles/DL) ===
product_name   model  N points  sMAPE_eps (%)  WAPE_eps (%)         RMSE  RMSLE (log1p)
     MacBook DL_Best         3      13.708372     12.891295  5932.827612       0.163323
        iPad DL_Best         3      28.659313     23.787879 11412.719232       0.347445
      iPhone DL_Best         3      11.239958     10.658259 15167.921871       0.120909

=== sMAPE_eps matrix (rows=product, cols=model) ===
model         DL_Best
product_name        

In [122]:
###Exporting a 12-month forecast for 2025 (classic + DL)
# You already have H from: model, best_kind, x_sc, y_sc, L, m_in, H, table = run_model_zoo(...)
H_EVAL = HORIZON 
# === Small helper so older code that calls `forecast_prophet` still works ===
def forecast_prophet(models, df, scen, horizon, products=PRODUCTS):
    """
    Thin wrapper around forecast_prophet_safe to match older call signatures.
    """
    return forecast_prophet_safe(
        models,
        df,
        scen=scen,
        horizon=horizon,
        products=products,
        regressors=["flag_holiday"]   # or [] if you don't want regressors
    )


dl_fc = forecast_baseline_for_scenario(
    df_use, model, x_sc, y_sc, scen="Baseline_Seasonal",
    horizon=H_EVAL,    # <- use the trained horizon (3), not 12
    lookback=LOOKBACK
)
 # columns: scenario, horizon(1..12), iPhone, iPad, MacBook

# 2) Classic models (these ignore window counts and can also do 12 months)
xgb_fc     = forecast_xgb(xgb_models, df_use, scen="Baseline_Seasonal", lookback=LOOKBACK, horizon=H_EVAL)
sarimax_fc = sarimax_forecast_df(sarimax_models, df_use, scen="Baseline_Seasonal", horizon=H_EVAL)
prophet_fc = forecast_prophet(prophet_models, df_use, scen="Baseline_Seasonal", horizon=H_EVAL)

# 3) Ensembles (optional)
ens_equal    = blend_forecasts([xgb_fc, sarimax_fc, prophet_fc])                       # equal weights
ens_weighted = blend_forecasts([xgb_fc, sarimax_fc, prophet_fc], weights=[0.4,0.2,0.4]) # example

# 4) Map horizon -> 2025 months and export a Word/CSV-friendly table
def add_2025_calendar(df_fc):
    out = df_fc.copy()
    out["Year"]  = 2025
    out["Month"] = out["horizon"]        # horizon 1..12 → Jan..Dec
    # Long format for manuscript / robust model
    long = out.melt(
        id_vars=["Year","Month","scenario","horizon"],
        value_vars=PRODUCTS,
        var_name="product_name", value_name="forecast_units"
    ).drop(columns=["horizon"])
    return long.sort_values(["product_name","Month"])

dl_2025       = add_2025_calendar(dl_fc)
xgb_2025      = add_2025_calendar(xgb_fc)
sarimax_2025  = add_2025_calendar(sarimax_fc)
prophet_2025  = add_2025_calendar(prophet_fc)
ens_eq_2025   = add_2025_calendar(ens_equal)
ens_w_2025    = add_2025_calendar(ens_weighted)

# Save what you’ll feed the robust model:
ens_w_2025.to_csv("forecast_2025_ensemble_weighted.csv", index=False)


In [129]:
# Build the aligned last-fold true vs. predicted arrays (H×m)
# Use the SAME horizon the classic models were trained with (currently 3)
H_EVAL = HORIZON   # this should be 3 in your config when they were trained

arrs = last_fold_arrays(
    {"XGB": xgb_models, "SARIMAX": sarimax_models, "Prophet": prophet_models},
    df_use, scen="Baseline_Seasonal", lookback=LOOKBACK, horizon=H_EVAL
)
# Optionally add DL:
dl_arr = dl_fc[PRODUCTS].to_numpy(dtype="float64")  # (12×3) for last forecast block
# If you want DL residuals on last-fold, compute a DL forecast *at the last fold*, not the future.

# Make a residual table (per product, per horizon)
def residual_table(y_true, y_pred, model_name):
    H, m = y_true.shape
    rows = []
    for j, prod in enumerate(PRODUCTS):
        for h in range(H):
            err = y_true[h, j] - y_pred[h, j]
            rows.append({"product_name": prod, "horizon": h+1, "error": float(err), "model": model_name})
    return pd.DataFrame(rows)

y_true = arrs["true"]
res_xgb = residual_table(y_true, arrs["XGB"], "XGB")
res_prop= residual_table(y_true, arrs["Prophet"], "Prophet")
res_sar = residual_table(y_true, arrs["SARIMAX"], "SARIMAX")

residuals = pd.concat([res_xgb, res_prop, res_sar], ignore_index=True)
# Summaries per product, per horizon (mean/quantiles for DDRO bands)
err_summary = (residuals
    .groupby(["product_name","horizon","model"])["error"]
    .agg(["mean","std", lambda s: s.quantile(0.1), lambda s: s.quantile(0.9)])
    .rename(columns={"<lambda_0>":"q10","<lambda_1>":"q90"})
    .reset_index()
)
err_summary.to_csv("error_bands_lastfold_per_product_h.csv", index=False)


Running forecasts for scenario 'Baseline_Seasonal' using t_end=56

--- Sanity check: median/mean comparison ---
true     median=   40772.683 mean=   69255.273 | ratio= 1.00
Prophet  median=   40417.191 mean=   66453.488 | ratio= 0.99
SARIMAX  median=   37732.883 mean=   60548.037 | ratio= 0.93
XGB      median=   40763.348 mean=   69277.587 | ratio= 1.00


In [125]:
import numpy as np
import pandas as pd

def export_sarimax_residuals(models, df, scen, products, outfile):
    """
    Build a CSV of in-sample one-step-ahead residuals for SARIMAX,
    for a single scenario.

    Each row: product_name, time_index, Year, period, month, ds, actual, forecast, error
    """
    # 1) Standardize and add t_month
    d = make_df_use(df)                 # your existing helper
    d = normalize_time_strict(d)        # adds t_month per scenario

    rows = []

    for prod in products:
        key = (scen, prod)
        if key not in models:
            print(f"[WARN] SARIMAX model missing for {key}, skipping.")
            continue

        res_info = models[key]
        res     = res_info["res"]       # SARIMAXResults
        # If you stored log1p flag, you can read res_info["log1p"] here if needed

        g = (d[(d["scenario"] == scen) & (d["product_name"] == prod)]
               .sort_values(["t_month"])
               .reset_index(drop=True))

        # Actual series
        y = g["demand_units"].to_numpy(dtype=float)

        # In-sample prediction for the same length
        # (dynamic=False → true one-step-ahead where possible)
        pred_res = res.get_prediction(start=0, end=len(y)-1, dynamic=False)
        yhat = np.asarray(pred_res.predicted_mean, dtype=float)

        # Some early points may be NaN because of differencing; drop them
        for k in range(len(y)):
            if np.isnan(yhat[k]):
                continue

            rows.append({
                "product_name": prod,
                "time_index": int(g.loc[k, "t_month"]),
                "Year": int(g.loc[k, "Year"]),
                "period": int(g.loc[k, "period"]),
                "month": int(g.loc[k, "period"]),
                "ds": pd.Timestamp(year=int(g.loc[k, "Year"]),
                                   month=int(g.loc[k, "period"]),
                                   day=1).strftime("%Y-%m-%d"),
                "actual": float(y[k]),
                "forecast": float(yhat[k]),
                "error": float(y[k] - yhat[k]),
            })

    df_err = pd.DataFrame(rows).sort_values(["product_name","Year","period"])
    df_err.to_csv(outfile, index=False)
    print(f"Saved {outfile} with {len(df_err)} rows "
          f"for scenario '{scen}' and products={products}")
    return df_err


In [126]:
SCEN = "Baseline_Seasonal"   # same as before

sarimax_errors = export_sarimax_residuals(
    models=sarimax_models,   # dict[(scenario, product)] → {"res": SARIMAXResults, ...}
    df=df,                   # or df_use, whichever you used to fit
    scen=SCEN,
    products=PRODUCTS,
    outfile="sarimax_residuals_baseline_Adjusted.csv"
)


Saved sarimax_residuals_baseline_Adjusted.csv with 180 rows for scenario 'Baseline_Seasonal' and products=['iPhone', 'iPad', 'MacBook']


In [127]:
sarimax_errors

,product_name,time_index,Year,period,month,ds,actual,forecast,error
120,MacBook,0,2020,1,1,2020-01-01,31116.0000,0.000000,31116.000000
121,MacBook,1,2020,2,2,2020-02-01,29791.0000,31116.000000,-1325.000000
122,MacBook,2,2020,3,3,2020-03-01,32161.0000,29791.000000,2370.000000
123,MacBook,3,2020,4,4,2020-04-01,32656.0000,32161.000000,495.000000
124,MacBook,4,2020,5,5,2020-05-01,32794.0000,32656.000000,138.000000
...,...,...,...,...,...,...,...,...,...
55,iPhone,55,2024,8,8,2024-08-01,114390.4926,114374.959248,15.533352
56,iPhone,56,2024,9,9,2024-09-01,122971.7826,122945.909131,25.873469
57,iPhone,57,2024,10,10,2024-10-01,126492.4327,126481.840534,10.592166
58,iPhone,58,2024,11,11,2024-11-01,135728.8369,135700.981000,27.855900
